# Let us play: Create an LLM that can translate to punjabi
## Use Google Colab: Free T4 GPU, which is sufficient to run the model

In [1]:
## Run to install the desired libraries
# !pip install peft==0.8.2
# !pip install datasets==2.16.1

# We will use a simple 1b model which can fit in 15GB free TPU memory

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
import torch

model_name="bigscience/bloom-1b1"

tokenizer = AutoTokenizer.from_pretrained(model_name)
foundation_model = AutoModelForCausalLM.from_pretrained(model_name)

# Move model to GPU
foundation_model.to("cuda")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


BloomForCausalLM(
  (transformer): BloomModel(
    (word_embeddings): Embedding(250880, 1536)
    (word_embeddings_layernorm): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
    (h): ModuleList(
      (0-23): 24 x BloomBlock(
        (input_layernorm): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
        (self_attention): BloomAttention(
          (query_key_value): Linear(in_features=1536, out_features=4608, bias=True)
          (dense): Linear(in_features=1536, out_features=1536, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (post_attention_layernorm): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
        (mlp): BloomMLP(
          (dense_h_to_4h): Linear(in_features=1536, out_features=6144, bias=True)
          (gelu_impl): BloomGelu()
          (dense_4h_to_h): Linear(in_features=6144, out_features=1536, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
  )
  (

# Let us see the output of the base model

In [3]:
def get_outputs(model, inputs, max_new_tokens=100):
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=max_new_tokens,
        repetition_penalty=1.5,  # We don't want the repetition to happen
        early_stopping=True,  # Stop before reaching max_length
        eos_token_id=tokenizer.eos_token_id,
        num_beams=3
    )
    return outputs

# Inference original model
input_sentences = tokenizer("How are you?", return_tensors="pt")

# Move inputs to GPU
input_sentences = {k: v.to("cuda") for k, v in input_sentences.items()}

foundational_outputs_sentence = get_outputs(foundation_model, input_sentences, max_new_tokens=100)

print(tokenizer.batch_decode(foundational_outputs_sentence, skip_special_tokens=True))

['How are you?”\n\n“Good.”\n\n“And you?”\n\n“Good.”\n\n“And you?”\n\n“Good.”\n\n“And you?”\n\n“Good.”\n\n“And you?”\n\n“Good.”\n\n“And you?”\n\n“Good.”\n\n“And you?”\n\n“Good.”\n\n“And you?”\n\n“Good.”\n\n“And you?”\n\n“Good.”\n\n“And you?”']


# Let use some custom dataset to train LLM for punjabi translation

In [4]:
# Custom dataset for translation
import pandas as pd
from datasets import Dataset

In [5]:
custom_prompts = [
    {
        "prompt": "I want you to act as a linux terminal. ਮੈਂ ਚਾਹੁੰਦਾ ਹਾਂ ਕਿ ਤੁਸੀਂ ਇੱਕ ਲਿਨਕਸ ਟਰਮੀਨਲ ਵਜੋਂ ਕੰਮ ਕਰੋ।"
    },
    {
        "prompt": "Explain the concept of machine learning. ਮਸ਼ੀਨ ਲਰਨਿੰਗ ਦੇ ਅਸੂਲ ਨੂੰ ਸਮਝਾਓ।"
    },
    {
        "prompt": "Translate this sentence into Spanish. ਇਹ ਵਾਕ ਨੂੰ ਸਪੈਨਿਸ਼ ਵਿੱਚ ਅਨੁਵਾਦ ਕਰੋ।"
    },
    {
        "prompt": "Create a summary of this text. ਇਸ ਪਾਠ ਦਾ ਇਕ ਸੰਖੇਪ ਬਣਾਓ।"
    },
    {
        "prompt": "Generate a list of 5 healthy foods. 5 ਸਿਹਤਮੰਦ ਖਾਣੇ ਦੀ ਸੂਚੀ ਬਣਾਓ।"
    },
    {
        "prompt": "Describe the process of photosynthesis. ਫੋਟੋਸਿੰਥੈਸਿਸ ਦੀ ਪ੍ਰਕਿਰਿਆ ਦਾ ਵਰਣਨ ਕਰੋ।"
    },
    {
        "prompt": "What are the benefits of regular exercise? ਨਿਯਮਿਤ ਵਰਜ਼ਿਸ਼ ਦੇ ਫਾਇਦੇ ਕੀ ਹਨ?"
    },
    {
        "prompt": "Find the capital city of France. ਫ੍ਰਾਂਸ ਦੀ ਰਾਜਧਾਨੀ ਸ਼ਹਿਰ ਲੱਭੋ।"
    },
    {
        "prompt": "How do you solve a quadratic equation? ਤੁਸੀਂ ਇੱਕ ਚਤੁਰਭੁਜ ਸਮੀਕਰਨ ਨੂੰ ਕਿਵੇਂ ਹੱਲ ਕਰਦੇ ਹੋ?"
    },
    {
        "prompt": "List the top 10 programming languages in 2024. 2024 ਵਿੱਚ ਸਿਖਰ ਦੇ 10 ਪ੍ਰੋਗ੍ਰਾਮਿੰਗ ਭਾਸ਼ਾਵਾਂ ਦੀ ਸੂਚੀ ਦਿਓ।"
    },
    {
        "prompt": "Write a short story about a talking cat. ਇੱਕ ਬੋਲਦੇ ਬਿੱਲੀ ਬਾਰੇ ਇੱਕ ਛੋਟੀ ਕਹਾਣੀ ਲਿਖੋ।"
    },
    {
        "prompt": "Explain the theory of relativity in simple terms. ਰਿਲੇਟਿਵਿਟੀ ਦੇ ਸਿਧਾਂਤ ਨੂੰ ਸਧਾਰਨ ਸ਼ਬਦਾਂ ਵਿੱਚ ਸਮਝਾਓ।"
    },
    {
        "prompt": "Generate a poem about the moonlight. ਚੰਦਨੀ ਬਾਰੇ ਇੱਕ ਕਵਿਤਾ ਬਣਾਓ।"
    },
    {
        "prompt": "What are some good pickup lines? ਕੁਝ ਵਧੀਆ ਪਿਕਅੱਪ ਲਾਈਨਾਂ ਕੀ ਹਨ?"
    },
    {
        "prompt": "Describe the perfect vacation spot. ਸਬ ਤੋਂ ਵਧੀਆ ਛੁੱਟੀਆਂ ਵਾਲੀ ਥਾਂ ਦਾ ਵਰਣਨ ਕਰੋ।"
    },
    {
        "prompt": "What are the top 10 ways to stay productive? ਉਤਪਾਦਕ ਬਣੇ ਰਹਿਣ ਦੇ ਸਿਖਰ ਦੇ 10 ਤਰੀਕੇ ਕੀ ਹਨ?"
    },
    {
        "prompt": "Create a dialogue between a robot and a human. ਰੋਬੋਟ ਅਤੇ ਮਨੁੱਖ ਦਰਮਿਆਨ ਇੱਕ ਸੰਵਾਦ ਬਣਾਓ।"
    },
    {
        "prompt": "Summarize the plot of 'Hamlet'. 'ਹੈਮਲੇਟ' ਦੀ ਕਥਾ ਦਾ ਸੰਖੇਪ ਦਿਓ।"
    },
    {
        "prompt": "Write a motivational speech for students. ਵਿਦਿਆਰਥੀਆਂ ਲਈ ਇੱਕ ਪ੍ਰੇਰਣਾਤਮਕ ਭਾਸ਼ਣ ਲਿਖੋ।"
    },
    {
        "prompt": "Explain the basics of blockchain technology. ਬਲੌਕਚੇਨ ਤਕਨਾਲੋਜੀ ਦੇ ਮੂਲ ਤੱਤ ਸਮਝਾਓ।"
    },
    {
        "prompt": "Describe the lifecycle of a butterfly. ਇੱਕ ਤਿਤਲੀ ਦੀ ਜ਼ਿੰਦਗੀ ਚੱਕਰ ਦਾ ਵਰਣਨ ਕਰੋ।"
    },
    {
        "prompt": "What are the health benefits of yoga? ਯੋਗ ਦੇ ਸਿਹਤ ਲਾਭ ਕੀ ਹਨ?"
    },
    {
        "prompt": "Explain the process of photosynthesis. ਫੋਟੋਸਿੰਥੈਸਿਸ ਦੀ ਪ੍ਰਕਿਰਿਆ ਸਮਝਾਓ।"
    },
    {
        "prompt": "Describe the function of the heart. ਦਿਲ ਦੀ ਕਾਰਜ ਸ਼ੀਲਤਾ ਦਾ ਵਰਣਨ ਕਰੋ।"
    },
    {
        "prompt": "What are the main causes of climate change? ਮੌਸਮ ਪਰੀਵਰਤਨ ਦੇ ਮੁੱਖ ਕਾਰਨ ਕੀ ਹਨ?"
    },
    {
        "prompt": "Create a list of the 7 wonders of the world. ਦੁਨੀਆ ਦੇ 7 ਅਜੂਬਿਆਂ ਦੀ ਸੂਚੀ ਬਣਾਓ।"
    },
    {
        "prompt": "Explain the importance of sleep. ਨੀਂਦ ਦੀ ਮਹੱਤਤਾ ਸਮਝਾਓ।"
    },
    {
        "prompt": "What are the symptoms of the common cold? ਆਮ ਜ਼ੁਕਾਮ ਦੇ ਲੱਛਣ ਕੀ ਹਨ?"
    },
    {
        "prompt": "Describe the process of water filtration. ਪਾਣੀ ਦੇ ਫਿਲਟ੍ਰੇਸ਼ਨ ਦੀ ਪ੍ਰਕਿਰਿਆ ਦਾ ਵਰਣਨ ਕਰੋ।"
    },
    {
        "prompt": "Explain the significance of the Great Wall of China. ਚੀਨ ਦੀ ਮਹਾਨ ਦਿਵਾਰ ਦੀ ਮਹੱਤਤਾ ਸਮਝਾਓ।"
    },
    {
        "prompt": "Generate a creative story about a haunted house. ਇੱਕ ਭੂਤ ਬੰਗਲੇ ਬਾਰੇ ਇੱਕ ਰਚਨਾਤਮਕ ਕਹਾਣੀ ਬਣਾਓ।"
    },
    {
        "prompt": "Describe the key features of a smartphone. ਇੱਕ ਸਮਾਰਟਫੋਨ ਦੀ ਮੁੱਖ ਵਿਸ਼ੇਸ਼ਤਾਵਾਂ ਦਾ ਵਰਣਨ ਕਰੋ।"
    },
    {
        "prompt": "What are the benefits of a balanced diet? ਸੰਤੁਲਿਤ ਖੁਰਾਕ ਦੇ ਫਾਇਦੇ ਕੀ ਹਨ?"
    },
    {
        "prompt": "Explain the water cycle. ਪਾਣੀ ਦੇ ਚੱਕਰ ਨੂੰ ਸਮਝਾਓ।"
    },
    {
        "prompt": "What is the significance of the Taj Mahal? ਤਾਜ ਮਹਲ ਦੀ ਮਹੱਤਤਾ ਕੀ ਹੈ?"
    },
    {
        "prompt": "Generate a list of 10 famous inventors. 10 ਮਸ਼ਹੂਰ ਖੋਜਕਾਰਾਂ ਦੀ ਸੂਚੀ ਬਣਾਓ।"
    },
    {
        "prompt": "Describe the process of making chocolate. ਚਾਕਲੇਟ ਬਣਾਉਣ ਦੀ ਪ੍ਰਕਿਰਿਆ ਦਾ ਵਰਣਨ ਕਰੋ।"
    },
    {
        "prompt": "Explain the importance of education. ਸਿੱਖਿਆ ਦੀ ਮਹੱਤਤਾ ਸਮਝਾਓ।"
    },
    {
        "prompt": "What are the different types of renewable energy? ਨਵੀਨੀਕਰਣ ਯੋਗ ਊਰਜਾ ਦੇ ਵੱਖ ਵੱਖ ਤਰੀਕੇ ਕੀ ਹਨ?"
    },
    {
        "prompt": "Describe the anatomy of the human brain. ਮਨੁੱਖੀ ਦਿਮਾਗ ਦੀ ਬਣਾਵਟ ਦਾ ਵਰਣਨ ਕਰੋ।"
    },
    {
        "prompt": "What are the top 5 tourist destinations in India? ਭਾਰਤ ਵਿੱਚ ਸਿਖਰ ਦੇ 5 ਸੈਲਾਨੀ ਮੰਜ਼ਿਲਾਂ ਕੀ ਹਨ?"
    },
    {
        "prompt": "Explain the process of cell division. ਸੈੱਲ ਵਿਭਾਜਨ ਦੀ ਪ੍ਰਕਿਰਿਆ ਸਮਝਾਓ।"
    },
    {
        "prompt": "Describe the role of the Prime Minister in a country. ਕਿਸੇ ਦੇਸ਼ ਵਿੱਚ ਪ੍ਰਧਾਨ ਮੰਤਰੀ ਦੀ ਭੂਮਿਕਾ ਦਾ ਵਰਣਨ ਕਰੋ।"
    },
    {
        "prompt": "What are the benefits of meditation? ਧਿਆਨ ਦੇ ਫਾਇਦੇ ਕੀ ਹਨ?"
    },
    {
        "prompt": "Generate a list of the 10 largest animals on Earth. ਧਰਤੀ ਦੇ 10 ਸਭ ਤੋਂ ਵੱਡੇ ਜਾਨਵਰਾਂ ਦੀ ਸੂਚੀ ਬਣਾਓ।"
    },
    {
        "prompt": "Describe the main events of World War II. ਦੂਜੀ ਵਿਸ਼ਵ ਯੁੱਧ ਦੇ ਮੁੱਖ ਘਟਨਾਵਾਂ ਦਾ ਵਰਣਨ ਕਰੋ।"
    },
    {
        "prompt": "Explain the concept of artificial intelligence. ਕ੍ਰਿਤ੍ਰਿਮ ਬੁੱਧਿਮੱਤਾ ਦੇ ਅਸੂਲ ਨੂੰ ਸਮਝਾਓ।"
    },
    {
        "prompt": "What are the different types of ecosystems? ਵੱਖ ਵੱਖ ਤਰ੍ਹਾਂ ਦੇ ਪਰਿਸ਼ਥਿਤਿਕੀ ਤੰਦਰੁਸਤੀ ਸਿਸਟਮ ਕੀ ਹਨ?"
    },
    {
        "prompt": "Describe the cultural significance of Diwali. ਦਿਵਾਲੀ ਦੀ ਸੱਭਿਆਚਾਰਕ ਮਹੱਤਤਾ ਦਾ ਵਰਣਨ ਕਰੋ।"
    },
    {
        "prompt": "Explain the process of volcanic eruption. ਜਵਾਲਾਮੁਖੀ ਵਿਸਫੋਟ ਦੀ ਪ੍ਰਕਿਰਿਆ ਸਮਝਾਓ।"
    },
    {
        "prompt": "What are the health benefits of drinking water? ਪਾਣੀ ਪੀਣ ਦੇ ਸਿਹਤ ਲਾਭ ਕੀ ਹਨ?"
    },
    {
        "prompt": "Describe the history of the Internet. ਇੰਟਰਨੈਟ ਦੀ ਇਤਿਹਾਸ ਦਾ ਵਰਣਨ ਕਰੋ।"
    },
    {
        "prompt": "Explain the principles of democracy. ਲੋਕਤੰਤਰ ਦੇ ਸਿਧਾਂਤ ਸਮਝਾਓ।"
    },
    {
        "prompt": "What are the main components of a computer? ਕੰਪਿਊਟਰ ਦੇ ਮੁੱਖ ਹਿੱਸੇ ਕੀ ਹਨ?"
    },
    {
        "prompt": "Describe the process of making bread. ਬ੍ਰੇਡ ਬਣਾਉਣ ਦੀ ਪ੍ਰਕਿਰਿਆ ਦਾ ਵਰਣਨ ਕਰੋ।"
    },
    {
        "prompt": "Explain the significance of the Eiffel Tower. ਆਈਫਲ ਟਾਵਰ ਦੀ ਮਹੱਤਤਾ ਸਮਝਾਓ।"
    },
    {
        "prompt": "Generate a list of the top 10 books of all time. ਸਭ ਸਮੇਂ ਦੇ ਸਿਖਰ ਦੇ 10 ਕਿਤਾਬਾਂ ਦੀ ਸੂਚੀ ਬਣਾਓ।"
    },
    {
        "prompt": "Describe the stages of human development. ਮਨੁੱਖੀ ਵਿਕਾਸ ਦੇ ਪੜਾਅ ਦਾ ਵਰਣਨ ਕਰੋ।"
    },
    {
        "prompt": "What are the benefits of learning a second language? ਦੂਜੀ ਭਾਸ਼ਾ ਸਿੱਖਣ ਦੇ ਫਾਇਦੇ ਕੀ ਹਨ?"
    },
    {
        "prompt": "Explain the process of recycling. ਰੀਸਾਈਕਲਿੰਗ ਦੀ ਪ੍ਰਕਿਰਿਆ ਸਮਝਾਓ।"
    },
    {
        "prompt": "Describe the features of the Amazon rainforest. ਅਮੇਜ਼ਨ ਵਰਖਾ ਜੰਗਲ ਦੀਆਂ ਵਿਸ਼ੇਸ਼ਤਾਵਾਂ ਦਾ ਵਰਣਨ ਕਰੋ।"
    },
    {
        "prompt": "What are the top 5 benefits of exercise? ਕਸਰਤ ਦੇ ਸਿਖਰ ਦੇ 5 ਫਾਇਦੇ ਕੀ ਹਨ?"
    },
    {
        "prompt": "Explain the theory of evolution. ਵਿਕਾਸ ਦੇ ਸਿਧਾਂਤ ਨੂੰ ਸਮਝਾਓ।"
    },
    {
        "prompt": "Describe the main functions of the liver. ਜਿਗਰ ਦੀਆਂ ਮੁੱਖ ਕਾਰਜਾਂ ਦਾ ਵਰਣਨ ਕਰੋ।"
    },
    {
        "prompt": "What are the different types of pollution? ਪ੍ਰਦੂਸ਼ਣ ਦੇ ਵੱਖ ਵੱਖ ਤਰੀਕੇ ਕੀ ਹਨ?"
    },
    {
        "prompt": "Explain the significance of the Great Barrier Reef. ਮਹਾਨ ਬੈਰੀਅਰ ਰੀਫ ਦੀ ਮਹੱਤਤਾ ਸਮਝਾਓ।"
    },
    {
        "prompt": "Generate a list of the 10 smallest countries in the world. ਦੁਨੀਆ ਦੇ 10 ਸਭ ਤੋਂ ਛੋਟੇ ਦੇਸ਼ਾਂ ਦੀ ਸੂਚੀ ਬਣਾਓ।"
    },
    {
        "prompt": "Describe the process of fermentation. ਖਮੀਰਕਰਨ ਦੀ ਪ੍ਰਕਿਰਿਆ ਦਾ ਵਰਣਨ ਕਰੋ।"
    },
    {
        "prompt": "What are the benefits of using renewable energy sources? ਨਵੀਨੀਕਰਣ ਯੋਗ ਊਰਜਾ ਸ੍ਰੋਤਾਂ ਦੇ ਫਾਇਦੇ ਕੀ ਹਨ?"
    },
    {
        "prompt": "Explain the importance of biodiversity. ਜੈਵ ਵਿਭਿੰਨਤਾ ਦੀ ਮਹੱਤਤਾ ਸਮਝਾਓ।"
    }
]


In [6]:
custom_prompts2 = [
    {
        "prompt": "Write a review of your favorite book. ਆਪਣੀ ਮਨਪਸੰਦ ਕਿਤਾਬ ਦਾ ਸਮੀਖਿਆ ਲਿਖੋ।"
    },
    {
        "prompt": "Explain the concept of gravitational force. ਗੁਰੂਤਵਾਕਰਸ਼ਣ ਬਲ ਦੇ ਅਸੂਲ ਨੂੰ ਸਮਝਾਓ।"
    },
    {
        "prompt": "Translate this paragraph into French. ਇਸ ਪੈਰਾ ਦਾ ਫਰੈਂਚ ਵਿੱਚ ਅਨੁਵਾਦ ਕਰੋ।"
    },
    {
        "prompt": "Create a summary of a movie you recently watched. ਤੁਸੀਂ ਹਾਲ ਹੀ ਵਿੱਚ ਦੇਖੀ ਫਿਲਮ ਦਾ ਇਕ ਸੰਖੇਪ ਬਣਾਓ।"
    },
    {
        "prompt": "Generate a list of 5 benefits of reading books. ਕਿਤਾਬਾਂ ਪੜ੍ਹਨ ਦੇ 5 ਫਾਇਦੇ ਦੀ ਸੂਚੀ ਬਣਾਓ।"
    },
    {
        "prompt": "Describe the process of making tea. ਚਾਹ ਬਣਾਉਣ ਦੀ ਪ੍ਰਕਿਰਿਆ ਦਾ ਵਰਣਨ ਕਰੋ।"
    },
    {
        "prompt": "What are the benefits of a healthy lifestyle? ਸਿਹਤਮੰਦ ਜੀਵਨਸ਼ੈਲੀ ਦੇ ਫਾਇਦੇ ਕੀ ਹਨ?"
    },
    {
        "prompt": "Find the population of India. ਭਾਰਤ ਦੀ ਅਬਾਦੀ ਲੱਭੋ।"
    },
    {
        "prompt": "How do you calculate the area of a circle? ਤੁਸੀਂ ਗੋਲ ਦਾਅਰਾ ਕਿਵੇਂ ਗਣਨਾ ਕਰਦੇ ਹੋ?"
    },
    {
        "prompt": "List the top 10 universities in the world. ਦੁਨੀਆ ਦੀਆਂ ਸਿਖਰ ਦੀਆਂ 10 ਯੂਨੀਵਰਸਿਟੀਆਂ ਦੀ ਸੂਚੀ ਦਿਓ।"
    },
    {
        "prompt": "Write a poem about friendship. ਦੋਸਤੀ ਬਾਰੇ ਇੱਕ ਕਵਿਤਾ ਲਿਖੋ।"
    },
    {
        "prompt": "Explain the concept of renewable energy. ਨਵੀਨੀਕਰਣ ਯੋਗ ਊਰਜਾ ਦੇ ਅਸੂਲ ਨੂੰ ਸਮਝਾਓ।"
    },
    {
        "prompt": "Translate this recipe into Italian. ਇਸ ਰਸੋਈ ਦਾ ਇਤਾਲਵੀ ਵਿੱਚ ਅਨੁਵਾਦ ਕਰੋ।"
    },
    {
        "prompt": "Create a travel guide for Paris. ਪੈਰਿਸ ਲਈ ਇੱਕ ਯਾਤਰਾ ਗਾਈਡ ਬਣਾਓ।"
    },
    {
        "prompt": "Generate a list of 5 popular programming frameworks. 5 ਪ੍ਰਸਿੱਧ ਪ੍ਰੋਗ੍ਰਾਮਿੰਗ ਫਰੇਮਵਰਕ ਦੀ ਸੂਚੀ ਬਣਾਓ।"
    },
    {
        "prompt": "Describe the process of making a website. ਇੱਕ ਵੈਬਸਾਈਟ ਬਣਾਉਣ ਦੀ ਪ੍ਰਕਿਰਿਆ ਦਾ ਵਰਣਨ ਕਰੋ।"
    },
    {
        "prompt": "What are the benefits of learning to code? ਕੋਡਿੰਗ ਸਿੱਖਣ ਦੇ ਫਾਇਦੇ ਕੀ ਹਨ?"
    },
    {
        "prompt": "Find the tallest mountain in the world. ਦੁਨੀਆ ਦਾ ਸਭ ਤੋਂ ਉੱਚਾ ਪਹਾੜ ਲੱਭੋ।"
    },
    {
        "prompt": "How do you solve a linear equation? ਤੁਸੀਂ ਇੱਕ ਸਧਾਰਨ ਸਮੀਕਰਨ ਨੂੰ ਕਿਵੇਂ ਹੱਲ ਕਰਦੇ ਹੋ?"
    },
    {
        "prompt": "List the top 10 tourist attractions in Japan. ਜਪਾਨ ਵਿੱਚ ਸਿਖਰ ਦੇ 10 ਸੈਲਾਨੀ ਆਕਰਸ਼ਣਾਂ ਦੀ ਸੂਚੀ ਦਿਓ।"
    },
    {
        "prompt": "Write a short story about a magical forest. ਇੱਕ ਜਾਦੂਈ ਜੰਗਲ ਬਾਰੇ ਇੱਕ ਛੋਟੀ ਕਹਾਣੀ ਲਿਖੋ।"
    },
    {
        "prompt": "Explain the concept of climate change. ਮੌਸਮ ਪਰੀਵਰਤਨ ਦੇ ਅਸੂਲ ਨੂੰ ਸਮਝਾਓ।"
    },
    {
        "prompt": "Translate this document into German. ਇਸ ਦਸਤਾਵੇਜ਼ ਨੂੰ ਜਰਮਨ ਵਿੱਚ ਅਨੁਵਾਦ ਕਰੋ।"
    },
    {
        "prompt": "Create a marketing plan for a new product. ਇੱਕ ਨਵੇਂ ਉਤਪਾਦ ਲਈ ਮਾਰਕੀਟਿੰਗ ਯੋਜਨਾ ਬਣਾਓ।"
    },
    {
        "prompt": "Generate a list of 5 effective study techniques. 5 ਪ੍ਰਭਾਵਸ਼ਾਲੀ ਅਧਿਐਨ ਤਕਨੀਕਾਂ ਦੀ ਸੂਚੀ ਬਣਾਓ।"
    },
    {
        "prompt": "Describe the process of writing a research paper. ਇੱਕ ਖੋਜ ਪੇਪਰ ਲਿਖਣ ਦੀ ਪ੍ਰਕਿਰਿਆ ਦਾ ਵਰਣਨ ਕਰੋ।"
    },
    {
        "prompt": "What are the benefits of exercise for mental health? ਮਾਨਸਿਕ ਸਿਹਤ ਲਈ ਕਸਰਤ ਦੇ ਫਾਇਦੇ ਕੀ ਹਨ?"
    },
    {
        "prompt": "Find the GDP of the United States. ਸੰਯੁਕਤ ਰਾਜ ਦੇ GDP ਨੂੰ ਲੱਭੋ।"
    },
    {
        "prompt": "How do you perform a financial analysis? ਤੁਸੀਂ ਇੱਕ ਆਰਥਿਕ ਵਿਸ਼ਲੇਸ਼ਣ ਕਿਵੇਂ ਕਰਦੇ ਹੋ?"
    },
    {
        "prompt": "List the top 10 largest cities in the world. ਦੁਨੀਆ ਦੇ ਸਿਖਰ ਦੇ 10 ਸਭ ਤੋਂ ਵੱਡੇ ਸ਼ਹਿਰਾਂ ਦੀ ਸੂਚੀ ਦਿਓ।"
    },
    {
        "prompt": "Write a letter to your future self. ਆਪਣੇ ਭਵਿੱਖ ਦੇ ਆਪ ਨੂੰ ਇੱਕ ਚਿੱਠੀ ਲਿਖੋ।"
    },
    {
        "prompt": "Explain the concept of blockchain. ਬਲੌਕਚੇਨ ਦੇ ਅਸੂਲ ਨੂੰ ਸਮਝਾਓ।"
    },
    {
        "prompt": "Translate this text into Chinese. ਇਸ ਪਾਠ ਦਾ ਚੀਨੀ ਵਿੱਚ ਅਨੁਵਾਦ ਕਰੋ।"
    },
    {
        "prompt": "Create a business plan for a startup. ਇੱਕ ਸਟਾਰਟਅਪ ਲਈ ਵਪਾਰ ਯੋਜਨਾ ਬਣਾਓ।"
    },
    {
        "prompt": "Generate a list of 5 innovative technologies. 5 ਨਵੀਆਂ ਤਕਨਾਲੋਜੀਆਂ ਦੀ ਸੂਚੀ ਬਣਾਓ।"
    },
    {
        "prompt": "Describe the process of learning a new language. ਇੱਕ ਨਵੀਂ ਭਾਸ਼ਾ ਸਿੱਖਣ ਦੀ ਪ੍ਰਕਿਰਿਆ ਦਾ ਵਰਣਨ ਕਰੋ।"
    },
    {
        "prompt": "What are the benefits of volunteering? ਸੇਵਾ ਕਰਣ ਦੇ ਫਾਇਦੇ ਕੀ ਹਨ?"
    },
    {
        "prompt": "Find the largest desert in the world. ਦੁਨੀਆ ਦਾ ਸਭ ਤੋਂ ਵੱਡਾ ਰੇਗਿਸਤਾਨ ਲੱਭੋ।"
    },
    {
        "prompt": "How do you create a budget plan? ਤੁਸੀਂ ਇੱਕ ਬਜਟ ਯੋਜਨਾ ਕਿਵੇਂ ਬਣਾਉਂਦੇ ਹੋ?"
    },
    {
        "prompt": "List the top 10 most spoken languages in the world. ਦੁਨੀਆ ਵਿੱਚ ਸਭ ਤੋਂ ਜ਼ਿਆਦਾ ਬੋਲੀਆਂ ਜਾਣ ਵਾਲੀਆਂ 10 ਭਾਸ਼ਾਵਾਂ ਦੀ ਸੂਚੀ ਦਿਓ।"
    },
    {
        "prompt": "Write a poem about nature. ਕੁਦਰਤ ਬਾਰੇ ਇੱਕ ਕਵਿਤਾ ਲਿਖੋ।"
    },
    {
        "prompt": "Explain the concept of artificial intelligence. ਕ੍ਰਿਤ੍ਰਿਮ ਬੁੱਧਿਮੱਤਾ ਦੇ ਅਸੂਲ ਨੂੰ ਸਮਝਾਓ।"
    },
    {
        "prompt": "Translate this passage into Japanese. ਇਸ ਪੈਸੇਜ ਦਾ ਜਪਾਨੀ ਵਿੱਚ ਅਨੁਵਾਦ ਕਰੋ।"
    },
    {
        "prompt": "Create a workout plan for beginners. ਸ਼ੁਰੂਆਤੀ ਲਈ ਇੱਕ ਵਰਕਆਉਟ ਯੋਜਨਾ ਬਣਾਓ।"
    },
    {
        "prompt": "Generate a list of 5 famous historical figures. 5 ਪ੍ਰਸਿੱਧ ਇਤਿਹਾਸਕ ਹਸਤੀ ਦੀ ਸੂਚੀ ਬਣਾਓ।"
    },
    {
        "prompt": "Describe the process of baking a cake. ਇੱਕ ਕੇਕ ਬਨਾਉਣ ਦੀ ਪ੍ਰਕਿਰਿਆ ਦਾ ਵਰਣਨ ਕਰੋ।"
    },
    {
        "prompt": "What are the benefits of traveling? ਯਾਤਰਾ ਕਰਨ ਦੇ ਫਾਇਦੇ ਕੀ ਹਨ?"
    },
    {
        "prompt": "Find the largest ocean in the world. ਦੁਨੀਆ ਦਾ ਸਭ ਤੋਂ ਵੱਡਾ ਮਹਾਂਸਾਗਰ ਲੱਭੋ।"
    },
    {
        "prompt": "How do you write a cover letter? ਤੁਸੀਂ ਇੱਕ ਕਵਰ ਲੈਟਰ ਕਿਵੇਂ ਲਿਖਦੇ ਹੋ?"
    },
    {
        "prompt": "List the top 10 highest mountains in the world. ਦੁਨੀਆ ਦੇ ਸਿਖਰ ਦੇ 10 ਸਭ ਤੋਂ ਉੱਚੇ ਪਹਾੜਾਂ ਦੀ ਸੂਚੀ ਦਿਓ।"
    }
]


In [7]:
custom_prompts3 = [
    {
        "prompt": "Write a review of your favorite movie. ਆਪਣੀ ਮਨਪਸੰਦ ਫਿਲਮ ਦੀ ਸਮੀਖਿਆ ਲਿਖੋ।"
    },
    {
        "prompt": "Explain the concept of electric cars. ਬਿਜਲੀ ਵਾਲੀਆਂ ਕਾਰਾਂ ਦੇ ਅਸੂਲ ਨੂੰ ਸਮਝਾਓ।"
    },
    {
        "prompt": "Translate this paragraph into Russian. ਇਸ ਪੈਰਾ ਦਾ ਰੂਸੀ ਵਿੱਚ ਅਨੁਵਾਦ ਕਰੋ।"
    },
    {
        "prompt": "Create a summary of your favorite TV show. ਆਪਣੀ ਮਨਪਸੰਦ ਟੀਵੀ ਸ਼ੋ ਦਾ ਸੰਖੇਪ ਬਣਾਓ।"
    },
    {
        "prompt": "Generate a list of 5 benefits of meditation. ਧਿਆਨ ਦੇ 5 ਫਾਇਦੇ ਦੀ ਸੂਚੀ ਬਣਾਓ।"
    },
    {
        "prompt": "Describe the process of making a smoothie. ਇੱਕ ਸਮੂਥੀ ਬਣਾਉਣ ਦੀ ਪ੍ਰਕਿਰਿਆ ਦਾ ਵਰਣਨ ਕਰੋ।"
    },
    {
        "prompt": "What are the benefits of a vegetarian diet? ਸ਼ਾਕਾਹਾਰੀ ਖੁਰਾਕ ਦੇ ਫਾਇਦੇ ਕੀ ਹਨ?"
    },
    {
        "prompt": "Find the capital city of Australia. ਆਸਟ੍ਰੇਲੀਆ ਦੀ ਰਾਜਧਾਨੀ ਸ਼ਹਿਰ ਲੱਭੋ।"
    },
    {
        "prompt": "How do you solve a system of linear equations? ਤੁਸੀਂ ਸਿਧਾਰਨ ਸਮੀਕਰਨਾਂ ਦੀ ਪ੍ਰਣਾਲੀ ਨੂੰ ਕਿਵੇਂ ਹੱਲ ਕਰਦੇ ਹੋ?"
    },
    {
        "prompt": "List the top 10 tourist attractions in Italy. ਇਟਲੀ ਵਿੱਚ ਸਿਖਰ ਦੇ 10 ਸੈਲਾਨੀ ਆਕਰਸ਼ਣਾਂ ਦੀ ਸੂਚੀ ਦਿਓ।"
    },
    {
        "prompt": "Write a poem about the ocean. ਮਹਾਂਸਾਗਰ ਬਾਰੇ ਇੱਕ ਕਵਿਤਾ ਲਿਖੋ।"
    },
    {
        "prompt": "Explain the concept of sustainable development. ਸਥਿਰ ਵਿਕਾਸ ਦੇ ਅਸੂਲ ਨੂੰ ਸਮਝਾਓ।"
    },
    {
        "prompt": "Translate this recipe into Portuguese. ਇਸ ਰਸੋਈ ਦਾ ਪੁਰਤਗਾਲੀ ਵਿੱਚ ਅਨੁਵਾਦ ਕਰੋ।"
    },
    {
        "prompt": "Create a travel guide for New York City. ਨਿਊਯਾਰਕ ਸ਼ਹਿਰ ਲਈ ਇੱਕ ਯਾਤਰਾ ਗਾਈਡ ਬਣਾਓ।"
    },
    {
        "prompt": "Generate a list of 5 programming languages for web development. ਵੈਬ ਵਿਕਾਸ ਲਈ 5 ਪ੍ਰੋਗ੍ਰਾਮਿੰਗ ਭਾਸ਼ਾਵਾਂ ਦੀ ਸੂਚੀ ਬਣਾਓ।"
    },
    {
        "prompt": "Describe the process of making a podcast. ਇੱਕ ਪਾਡਕਾਸਟ ਬਣਾਉਣ ਦੀ ਪ੍ਰਕਿਰਿਆ ਦਾ ਵਰਣਨ ਕਰੋ।"
    },
    {
        "prompt": "What are the benefits of online learning? ਆਨਲਾਈਨ ਸਿੱਖਣ ਦੇ ਫਾਇਦੇ ਕੀ ਹਨ?"
    },
    {
        "prompt": "Find the tallest building in the world. ਦੁਨੀਆ ਦੀ ਸਭ ਤੋਂ ਉੱਚੀ ਇਮਾਰਤ ਲੱਭੋ।"
    },
    {
        "prompt": "How do you calculate the volume of a cylinder? ਤੁਸੀਂ ਇੱਕ ਸਿਲਿੰਡਰ ਦਾ ਵਾਲਿਊਮ ਕਿਵੇਂ ਗਣਨਾ ਕਰਦੇ ਹੋ?"
    },
    {
        "prompt": "List the top 10 museums in the world. ਦੁਨੀਆ ਦੇ ਸਿਖਰ ਦੇ 10 ਅਜਾਇਬ ਘਰਾਂ ਦੀ ਸੂਚੀ ਦਿਓ।"
    },
    {
        "prompt": "Write a short story about an adventurous journey. ਇੱਕ ਸਫ਼ਰਮਈ ਯਾਤਰਾ ਬਾਰੇ ਇੱਕ ਛੋਟੀ ਕਹਾਣੀ ਲਿਖੋ।"
    },
    {
        "prompt": "Explain the concept of quantum computing. ਕੁਆੰਟਮ ਕੰਪਿਊਟਿੰਗ ਦੇ ਅਸੂਲ ਨੂੰ ਸਮਝਾਓ।"
    },
    {
        "prompt": "Translate this document into Arabic. ਇਸ ਦਸਤਾਵੇਜ਼ ਨੂੰ ਅਰਬੀ ਵਿੱਚ ਅਨੁਵਾਦ ਕਰੋ।"
    },
    {
        "prompt": "Create a marketing plan for a new restaurant. ਇੱਕ ਨਵੇਂ ਰੈਸਟੋਰੈਂਟ ਲਈ ਮਾਰਕੀਟਿੰਗ ਯੋਜਨਾ ਬਣਾਓ।"
    },
    {
        "prompt": "Generate a list of 5 famous scientists. 5 ਪ੍ਰਸਿੱਧ ਵਿਗਿਆਨੀਆਂ ਦੀ ਸੂਚੀ ਬਣਾਓ।"
    },
    {
        "prompt": "Describe the process of writing a novel. ਇੱਕ ਨਾਵਲ ਲਿਖਣ ਦੀ ਪ੍ਰਕਿਰਿਆ ਦਾ ਵਰਣਨ ਕਰੋ।"
    },
    {
        "prompt": "What are the benefits of practicing mindfulness? ਮਾਈੰਡਫੁਲਨੈਸ ਦਾ ਅਭਿਆਸ ਕਰਨ ਦੇ ਫਾਇਦੇ ਕੀ ਹਨ?"
    },
    {
        "prompt": "Find the GDP of China. ਚੀਨ ਦਾ GDP ਲੱਭੋ।"
    },
    {
        "prompt": "How do you create a financial plan? ਤੁਸੀਂ ਇੱਕ ਆਰਥਿਕ ਯੋਜਨਾ ਕਿਵੇਂ ਬਣਾਉਂਦੇ ਹੋ?"
    },
    {
        "prompt": "List the top 10 busiest airports in the world. ਦੁਨੀਆ ਦੇ ਸਭ ਤੋਂ ਰੁਸ਼ਰੋਸ਼ ਹਵਾਈ ਅੱਡਿਆਂ ਦੀ ਸੂਚੀ ਦਿਓ।"
    },
    {
        "prompt": "Write a letter to your best friend. ਆਪਣੇ ਸਭ ਤੋਂ ਵਧੀਆ ਦੋਸਤ ਨੂੰ ਇੱਕ ਚਿੱਠੀ ਲਿਖੋ।"
    },
    {
        "prompt": "Explain the concept of cryptocurrency. ਕ੍ਰਿਪਟੋਕਰੰਸੀ ਦੇ ਅਸੂਲ ਨੂੰ ਸਮਝਾਓ।"
    },
    {
        "prompt": "Translate this text into Korean. ਇਸ ਪਾਠ ਦਾ ਕੋਰੀਅਨ ਵਿੱਚ ਅਨੁਵਾਦ ਕਰੋ।"
    },
    {
        "prompt": "Create a business plan for an e-commerce website. ਇੱਕ ਈ-ਕਾਮਰਸ ਵੈਬਸਾਈਟ ਲਈ ਵਪਾਰ ਯੋਜਨਾ ਬਣਾਓ।"
    },
    {
        "prompt": "Generate a list of 5 important historical events. 5 ਮਹੱਤਵਪੂਰਨ ਇਤਿਹਾਸਕ ਘਟਨਾਵਾਂ ਦੀ ਸੂਚੀ ਬਣਾਓ।"
    },
    {
        "prompt": "Describe the process of learning to play an instrument. ਇੱਕ ਸਾਜ਼ ਵਜਾਉਣ ਦੀ ਸਿੱਖਣ ਦੀ ਪ੍ਰਕਿਰਿਆ ਦਾ ਵਰਣਨ ਕਰੋ।"
    },
    {
        "prompt": "What are the benefits of donating to charity? ਦਾਨ ਦੇਣ ਦੇ ਫਾਇਦੇ ਕੀ ਹਨ?"
    },
    {
        "prompt": "Find the longest river in the world. ਦੁਨੀਆ ਦੀ ਸਭ ਤੋਂ ਲੰਮੀ ਦਰਿਆ ਲੱਭੋ।"
    },
    {
        "prompt": "How do you create a project plan? ਤੁਸੀਂ ਇੱਕ ਪ੍ਰੋਜੈਕਟ ਯੋਜਨਾ ਕਿਵੇਂ ਬਣਾਉਂਦੇ ਹੋ?"
    },
    {
        "prompt": "List the top 10 most populated countries in the world. ਦੁਨੀਆ ਦੇ ਸਭ ਤੋਂ ਵੱਧ ਅਬਾਦੀ ਵਾਲੇ 10 ਦੇਸ਼ਾਂ ਦੀ ਸੂਚੀ ਦਿਓ।"
    },
    {
        "prompt": "Write a poem about love. ਪ੍ਰੇਮ ਬਾਰੇ ਇੱਕ ਕਵਿਤਾ ਲਿਖੋ।"
    },
    {
        "prompt": "Explain the concept of deep learning. ਡੀਪ ਲਰਨਿੰਗ ਦੇ ਅਸੂਲ ਨੂੰ ਸਮਝਾਓ।"
    },
    {
        "prompt": "Translate this passage into Hindi. ਇਸ ਪੈਸੇਜ ਦਾ ਹਿੰਦੀ ਵਿੱਚ ਅਨੁਵਾਦ ਕਰੋ।"
    },
    {
        "prompt": "Create a workout plan for advanced level. ਉੱਚ ਪੱਧਰ ਲਈ ਇੱਕ ਵਰਕਆਉਟ ਯੋਜਨਾ ਬਣਾਓ।"
    },
    {
        "prompt": "Generate a list of 5 renowned artists. 5 ਪ੍ਰਸਿੱਧ ਕਲਾਕਾਰਾਂ ਦੀ ਸੂਚੀ ਬਣਾਓ।"
    },
    {
        "prompt": "Describe the process of knitting a sweater. ਇੱਕ ਸਵੈਟਰ ਬੁਣਨ ਦੀ ਪ੍ਰਕਿਰਿਆ ਦਾ ਵਰਣਨ ਕਰੋ।"
    },
    {
        "prompt": "What are the benefits of a plant-based diet? ਪੌਧੇ ਆਧਾਰਿਤ ਖੁਰਾਕ ਦੇ ਫਾਇਦੇ ਕੀ ਹਨ?"
    },
    {
        "prompt": "Find the smallest country in the world. ਦੁਨੀਆ ਦਾ ਸਭ ਤੋਂ ਛੋਟਾ ਦੇਸ਼ ਲੱਭੋ।"
    },
    {
        "prompt": "How do you write a research proposal? ਤੁਸੀਂ ਇੱਕ ਖੋਜ ਪ੍ਰਸਤਾਵ ਕਿਵੇਂ ਲਿਖਦੇ ਹੋ?"
    },
    {
        "prompt": "List the top 10 most influential leaders in history. ਇਤਿਹਾਸ ਵਿੱਚ ਸਭ ਤੋਂ ਪ੍ਰਭਾਵਸ਼ਾਲੀ 10 ਆਗੂਆਂ ਦੀ ਸੂਚੀ ਦਿਓ।"
    }
]


In [8]:
custom_prompts4 = [
    {
        "prompt": "Write a review of your favorite restaurant. ਆਪਣੀ ਮਨਪਸੰਦ ਰੈਸਟੋਰੈਂਟ ਦੀ ਸਮੀਖਿਆ ਲਿਖੋ।"
    },
    {
        "prompt": "Explain the concept of renewable resources. ਨਵੀਨੀਕਰਣ ਯੋਗ ਸਾਧਨਾਂ ਦੇ ਅਸੂਲ ਨੂੰ ਸਮਝਾਓ।"
    },
    {
        "prompt": "Translate this paragraph into Japanese. ਇਸ ਪੈਰਾ ਦਾ ਜਪਾਨੀ ਵਿੱਚ ਅਨੁਵਾਦ ਕਰੋ।"
    },
    {
        "prompt": "Create a summary of your last vacation. ਆਪਣੀਆਂ ਆਖਰੀਆਂ ਛੁੱਟੀਆਂ ਦਾ ਇਕ ਸੰਖੇਪ ਬਣਾਓ।"
    },
    {
        "prompt": "Generate a list of 5 benefits of exercise. ਕਸਰਤ ਦੇ 5 ਫਾਇਦੇ ਦੀ ਸੂਚੀ ਬਣਾਓ।"
    },
    {
        "prompt": "Describe the process of making pasta. ਪਾਸਤਾ ਬਣਾਉਣ ਦੀ ਪ੍ਰਕਿਰਿਆ ਦਾ ਵਰਣਨ ਕਰੋ।"
    },
    {
        "prompt": "What are the benefits of a balanced diet? ਸੰਤੁਲਿਤ ਖੁਰਾਕ ਦੇ ਫਾਇਦੇ ਕੀ ਹਨ?"
    },
    {
        "prompt": "Find the population of Brazil. ਬ੍ਰਾਜ਼ੀਲ ਦੀ ਅਬਾਦੀ ਲੱਭੋ।"
    },
    {
        "prompt": "How do you solve a quadratic equation? ਤੁਸੀਂ ਇੱਕ ਚਤੁਰਭੁਜ ਸਮੀਕਰਨ ਨੂੰ ਕਿਵੇਂ ਹੱਲ ਕਰਦੇ ਹੋ?"
    },
    {
        "prompt": "List the top 10 places to visit in Europe. ਯੂਰਪ ਵਿੱਚ ਸੈਰ ਕਰਨ ਲਈ ਸਿਖਰ ਦੇ 10 ਸਥਾਨਾਂ ਦੀ ਸੂਚੀ ਦਿਓ।"
    },
    {
        "prompt": "Write a poem about the seasons. ਮੌਸਮਾਂ ਬਾਰੇ ਇੱਕ ਕਵਿਤਾ ਲਿਖੋ।"
    },
    {
        "prompt": "Explain the concept of the Internet of Things (IoT). ਇੰਟਰਨੈਟ ਆਫ਼ ਥਿੰਗਸ (IoT) ਦੇ ਅਸੂਲ ਨੂੰ ਸਮਝਾਓ।"
    },
    {
        "prompt": "Translate this recipe into Spanish. ਇਸ ਰਸੋਈ ਦਾ ਸਪੈਨਿਸ਼ ਵਿੱਚ ਅਨੁਵਾਦ ਕਰੋ।"
    },
    {
        "prompt": "Create a travel guide for London. ਲੰਡਨ ਲਈ ਇੱਕ ਯਾਤਰਾ ਗਾਈਡ ਬਣਾਓ।"
    },
    {
        "prompt": "Generate a list of 5 programming languages for mobile development. ਮੋਬਾਈਲ ਵਿਕਾਸ ਲਈ 5 ਪ੍ਰੋਗ੍ਰਾਮਿੰਗ ਭਾਸ਼ਾਵਾਂ ਦੀ ਸੂਚੀ ਬਣਾਓ।"
    },
    {
        "prompt": "Describe the process of creating a mobile app. ਇੱਕ ਮੋਬਾਈਲ ਐਪ ਬਣਾਉਣ ਦੀ ਪ੍ਰਕਿਰਿਆ ਦਾ ਵਰਣਨ ਕਰੋ।"
    },
    {
        "prompt": "What are the benefits of remote work? ਰਿਮੋਟ ਕੰਮ ਕਰਨ ਦੇ ਫਾਇਦੇ ਕੀ ਹਨ?"
    },
    {
        "prompt": "Find the highest waterfall in the world. ਦੁਨੀਆ ਦਾ ਸਭ ਤੋਂ ਉੱਚਾ ਝਰਨਾ ਲੱਭੋ।"
    },
    {
        "prompt": "How do you calculate the area of a rectangle? ਤੁਸੀਂ ਆਯਤ ਦਾ ਖੇਤਰਫਲ ਕਿਵੇਂ ਗਣਨਾ ਕਰਦੇ ਹੋ?"
    },
    {
        "prompt": "List the top 10 landmarks in the USA. USA ਦੇ ਸਿਖਰ ਦੇ 10 ਪ੍ਰਸਿੱਧ ਸਥਾਨਾਂ ਦੀ ਸੂਚੀ ਦਿਓ।"
    },
    {
        "prompt": "Write a short story about a magical adventure. ਇੱਕ ਜਾਦੂਈ ਸਫ਼ਰ ਬਾਰੇ ਇੱਕ ਛੋਟੀ ਕਹਾਣੀ ਲਿਖੋ।"
    },
    {
        "prompt": "Explain the concept of genetic engineering. ਜੈਵਿਕ ਇੰਜੀਨੀਅਰਿੰਗ ਦੇ ਅਸੂਲ ਨੂੰ ਸਮਝਾਓ।"
    },
    {
        "prompt": "Translate this document into French. ਇਸ ਦਸਤਾਵੇਜ਼ ਨੂੰ ਫਰੈਂਚ ਵਿੱਚ ਅਨੁਵਾਦ ਕਰੋ।"
    },
    {
        "prompt": "Create a marketing plan for a new tech gadget. ਇੱਕ ਨਵੇਂ ਤਕਨਾਲੋਜੀ ਗੈਜਿਟ ਲਈ ਮਾਰਕੀਟਿੰਗ ਯੋਜਨਾ ਬਣਾਓ।"
    },
    {
        "prompt": "Generate a list of 5 famous historical monuments. 5 ਪ੍ਰਸਿੱਧ ਇਤਿਹਾਸਕ ਸਮਾਰਕਾਂ ਦੀ ਸੂਚੀ ਬਣਾਓ।"
    },
    {
        "prompt": "Describe the process of writing a screenplay. ਇੱਕ ਸਕ੍ਰੀਨਪਲੇ ਲਿਖਣ ਦੀ ਪ੍ਰਕਿਰਿਆ ਦਾ ਵਰਣਨ ਕਰੋ।"
    },
    {
        "prompt": "What are the benefits of practicing yoga? ਯੋਗ ਦਾ ਅਭਿਆਸ ਕਰਨ ਦੇ ਫਾਇਦੇ ਕੀ ਹਨ?"
    },
    {
        "prompt": "Find the GDP of Canada. ਕੈਨੇਡਾ ਦਾ GDP ਲੱਭੋ।"
    },
    {
        "prompt": "How do you create a business proposal? ਤੁਸੀਂ ਇੱਕ ਵਪਾਰ ਪ੍ਰਸਤਾਵ ਕਿਵੇਂ ਬਣਾਉਂਦੇ ਹੋ?"
    },
    {
        "prompt": "List the top 10 tourist destinations in Asia. ਏਸ਼ੀਆ ਵਿੱਚ ਸਿਖਰ ਦੇ 10 ਸੈਲਾਨੀ ਮੰਜ਼ਿਲਾਂ ਦੀ ਸੂਚੀ ਦਿਓ।"
    },
    {
        "prompt": "Write a letter to your future employer. ਆਪਣੇ ਭਵਿੱਖ ਦੇ ਨਿਯੋਜਕ ਨੂੰ ਇੱਕ ਚਿੱਠੀ ਲਿਖੋ।"
    },
    {
        "prompt": "Explain the concept of machine learning. ਮਸ਼ੀਨ ਲਰਨਿੰਗ ਦੇ ਅਸੂਲ ਨੂੰ ਸਮਝਾਓ।"
    },
    {
        "prompt": "Translate this text into German. ਇਸ ਪਾਠ ਦਾ ਜਰਮਨ ਵਿੱਚ ਅਨੁਵਾਦ ਕਰੋ।"
    },
    {
        "prompt": "Create a business plan for a new app. ਇੱਕ ਨਵੇਂ ਐਪ ਲਈ ਵਪਾਰ ਯੋਜਨਾ ਬਣਾਓ।"
    },
    {
        "prompt": "Generate a list of 5 innovative startups. 5 ਨਵੀਨਤਮ ਸਟਾਰਟਅਪ ਦੀ ਸੂਚੀ ਬਣਾਓ।"
    },
    {
        "prompt": "Describe the process of learning to cook. ਪਕਾਉਣ ਦੀ ਸਿੱਖਣ ਦੀ ਪ੍ਰਕਿਰਿਆ ਦਾ ਵਰਣਨ ਕਰੋ।"
    },
    {
        "prompt": "What are the benefits of reading? ਪੜ੍ਹਨ ਦੇ ਫਾਇਦੇ ਕੀ ਹਨ?"
    },
    {
        "prompt": "Find the largest island in the world. ਦੁਨੀਆ ਦਾ ਸਭ ਤੋਂ ਵੱਡਾ ਟਾਪੂ ਲੱਭੋ।"
    },
    {
        "prompt": "How do you create a marketing strategy? ਤੁਸੀਂ ਇੱਕ ਮਾਰਕੀਟਿੰਗ ਰਣਨੀਤੀ ਕਿਵੇਂ ਬਣਾਉਂਦੇ ਹੋ?"
    },
    {
        "prompt": "List the top 10 cleanest cities in the world. ਦੁਨੀਆ ਦੇ ਸਭ ਤੋਂ ਸਾਫ਼ ਸ਼ਹਿਰਾਂ ਦੀ ਸੂਚੀ ਦਿਓ।"
    },
    {
        "prompt": "Write a poem about the stars. ਤਾਰਿਆਂ ਬਾਰੇ ਇੱਕ ਕਵਿਤਾ ਲਿਖੋ।"
    },
    {
        "prompt": "Explain the concept of virtual reality. ਵਰਚੁਅਲ ਰੀਐਲਟੀ ਦੇ ਅਸੂਲ ਨੂੰ ਸਮਝਾਓ।"
    },
    {
        "prompt": "Translate this passage into Italian. ਇਸ ਪੈਸੇਜ ਦਾ ਇਤਾਲਵੀ ਵਿੱਚ ਅਨੁਵਾਦ ਕਰੋ।"
    },
    {
        "prompt": "Create a fitness plan for beginners. ਸ਼ੁਰੂਆਤੀਆਂ ਲਈ ਇੱਕ ਫਿਟਨੈਸ ਯੋਜਨਾ ਬਣਾਓ।"
    },
    {
        "prompt": "Generate a list of 5 famous poets. 5 ਪ੍ਰਸਿੱਧ ਕਵੀਆਂ ਦੀ ਸੂਚੀ ਬਣਾਓ।"
    },
    {
        "prompt": "Describe the process of brewing coffee. ਕੌਫੀ ਬਣਾਉਣ ਦੀ ਪ੍ਰਕਿਰਿਆ ਦਾ ਵਰਣਨ ਕਰੋ।"
    },
    {
        "prompt": "What are the benefits of adopting a pet? ਪਾਲਤੂ ਜਾਨਵਰ ਨੂੰ ਗੋਦ ਲੈਣ ਦੇ ਫਾਇਦੇ ਕੀ ਹਨ?"
    },
    {
        "prompt": "Find the smallest continent in the world. ਦੁਨੀਆ ਦਾ ਸਭ ਤੋਂ ਛੋਟਾ ਮਹਾਦੀਪ ਲੱਭੋ।"
    },
    {
        "prompt": "How do you write a business report? ਤੁਸੀਂ ਇੱਕ ਵਪਾਰ ਰਿਪੋਰਟ ਕਿਵੇਂ ਲਿਖਦੇ ਹੋ?"
    },
    {
        "prompt": "List the top 10 richest people in the world. ਦੁਨੀਆ ਦੇ ਸਭ ਤੋਂ ਅਮੀਰ ਲੋਕਾਂ ਦੀ ਸੂਚੀ ਦਿਓ।"
    }
]


In [9]:
custom_prompts5 = [
    {
        "prompt": "Write a review of your favorite gadget. ਆਪਣੀ ਮਨਪਸੰਦ ਗੈਜਿਟ ਦੀ ਸਮੀਖਿਆ ਲਿਖੋ।"
    },
    {
        "prompt": "Explain the concept of cloud computing. ਕਲਾਉਡ ਕੰਪਿਊਟਿੰਗ ਦੇ ਅਸੂਲ ਨੂੰ ਸਮਝਾਓ।"
    },
    {
        "prompt": "Translate this paragraph into Italian. ਇਸ ਪੈਰਾ ਦਾ ਇਤਾਲਵੀ ਵਿੱਚ ਅਨੁਵਾਦ ਕਰੋ।"
    },
    {
        "prompt": "Create a summary of your favorite book series. ਆਪਣੀ ਮਨਪਸੰਦ ਕਿਤਾਬ ਸੀਰੀਜ਼ ਦਾ ਇਕ ਸੰਖੇਪ ਬਣਾਓ।"
    },
    {
        "prompt": "Generate a list of 5 benefits of drinking water. ਪਾਣੀ ਪੀਣ ਦੇ 5 ਫਾਇਦੇ ਦੀ ਸੂਚੀ ਬਣਾਓ।"
    },
    {
        "prompt": "Describe the process of making sushi. ਸੁਸ਼ੀ ਬਣਾਉਣ ਦੀ ਪ੍ਰਕਿਰਿਆ ਦਾ ਵਰਣਨ ਕਰੋ।"
    },
    {
        "prompt": "What are the benefits of a high-protein diet? ਉੱਚ ਪ੍ਰੋਟੀਨ ਖੁਰਾਕ ਦੇ ਫਾਇਦੇ ਕੀ ਹਨ?"
    },
    {
        "prompt": "Find the capital city of Germany. ਜਰਮਨੀ ਦੀ ਰਾਜਧਾਨੀ ਸ਼ਹਿਰ ਲੱਭੋ।"
    },
    {
        "prompt": "How do you calculate the area of a triangle? ਤੁਸੀਂ ਤਿਕੋਣ ਦਾ ਖੇਤਰਫਲ ਕਿਵੇਂ ਗਣਨਾ ਕਰਦੇ ਹੋ?"
    },
    {
        "prompt": "List the top 10 landmarks in South America. ਦੱਖਣੀ ਅਮਰੀਕਾ ਦੇ ਸਿਖਰ ਦੇ 10 ਪ੍ਰਸਿੱਧ ਸਥਾਨਾਂ ਦੀ ਸੂਚੀ ਦਿਓ।"
    },
    {
        "prompt": "Write a poem about autumn. ਪਤਝੜ ਬਾਰੇ ਇੱਕ ਕਵਿਤਾ ਲਿਖੋ।"
    },
    {
        "prompt": "Explain the concept of data science. ਡੇਟਾ ਸਾਇੰਸ ਦੇ ਅਸੂਲ ਨੂੰ ਸਮਝਾਓ।"
    },
    {
        "prompt": "Translate this recipe into French. ਇਸ ਰਸੋਈ ਦਾ ਫਰੈਂਚ ਵਿੱਚ ਅਨੁਵਾਦ ਕਰੋ।"
    },
    {
        "prompt": "Create a travel guide for Tokyo. ਟੋਕੀਓ ਲਈ ਇੱਕ ਯਾਤਰਾ ਗਾਈਡ ਬਣਾਓ।"
    },
    {
        "prompt": "Generate a list of 5 programming languages for data analysis. ਡੇਟਾ ਵਿਸ਼ਲੇਸ਼ਣ ਲਈ 5 ਪ੍ਰੋਗ੍ਰਾਮਿੰਗ ਭਾਸ਼ਾਵਾਂ ਦੀ ਸੂਚੀ ਬਣਾਓ।"
    },
    {
        "prompt": "Describe the process of creating a YouTube channel. ਇੱਕ ਯੂਟਿਊਬ ਚੈਨਲ ਬਣਾਉਣ ਦੀ ਪ੍ਰਕਿਰਿਆ ਦਾ ਵਰਣਨ ਕਰੋ।"
    },
    {
        "prompt": "What are the benefits of time management? ਸਮੇਂ ਦੇ ਪ੍ਰਬੰਧ ਦੇ ਫਾਇਦੇ ਕੀ ਹਨ?"
    },
    {
        "prompt": "Find the tallest tree in the world. ਦੁਨੀਆ ਦਾ ਸਭ ਤੋਂ ਉੱਚਾ ਰੁੱਖ ਲੱਭੋ।"
    },
    {
        "prompt": "How do you calculate the volume of a sphere? ਤੁਸੀਂ ਇੱਕ ਗੋਲਕ ਦਾ ਵਾਲਿਊਮ ਕਿਵੇਂ ਗਣਨਾ ਕਰਦੇ ਹੋ?"
    },
    {
        "prompt": "List the top 10 national parks in the world. ਦੁਨੀਆ ਦੇ ਸਿਖਰ ਦੇ 10 ਰਾਸ਼ਟਰੀ ਪਾਰਕਾਂ ਦੀ ਸੂਚੀ ਦਿਓ।"
    },
    {
        "prompt": "Write a short story about a space adventure. ਇੱਕ ਅੰਤਰਿਕਸ਼ ਯਾਤਰਾ ਬਾਰੇ ਇੱਕ ਛੋਟੀ ਕਹਾਣੀ ਲਿਖੋ।"
    },
    {
        "prompt": "Explain the concept of virtual assistants. ਵਰਚੁਅਲ ਸਹਾਇਕਾਂ ਦੇ ਅਸੂਲ ਨੂੰ ਸਮਝਾਓ।"
    },
    {
        "prompt": "Translate this document into Chinese. ਇਸ ਦਸਤਾਵੇਜ਼ ਨੂੰ ਚੀਨੀ ਵਿੱਚ ਅਨੁਵਾਦ ਕਰੋ।"
    },
    {
        "prompt": "Create a marketing plan for a new fashion brand. ਇੱਕ ਨਵੇਂ ਫੈਸ਼ਨ ਬ੍ਰਾਂਡ ਲਈ ਮਾਰਕੀਟਿੰਗ ਯੋਜਨਾ ਬਣਾਓ।"
    },
    {
        "prompt": "Generate a list of 5 influential world leaders. 5 ਪ੍ਰਭਾਵਸ਼ਾਲੀ ਵਿਸ਼ਵ ਆਗੂਆਂ ਦੀ ਸੂਚੀ ਬਣਾਓ।"
    },
    {
        "prompt": "Describe the process of writing a blog post. ਇੱਕ ਬਲੌਗ ਪੋਸਟ ਲਿਖਣ ਦੀ ਪ੍ਰਕਿਰਿਆ ਦਾ ਵਰਣਨ ਕਰੋ।"
    },
    {
        "prompt": "What are the benefits of regular sleep? ਨਿਯਮਿਤ ਨੀਂਦ ਦੇ ਫਾਇਦੇ ਕੀ ਹਨ?"
    },
    {
        "prompt": "Find the GDP of India. ਭਾਰਤ ਦਾ GDP ਲੱਭੋ।"
    },
    {
        "prompt": "How do you create a personal budget? ਤੁਸੀਂ ਇੱਕ ਨਿੱਜੀ ਬਜਟ ਕਿਵੇਂ ਬਣਾਉਂਦੇ ਹੋ?"
    },
    {
        "prompt": "List the top 10 destinations for a honeymoon. ਹਨੀਮੂਨ ਲਈ ਸਿਖਰ ਦੇ 10 ਮੰਜ਼ਿਲਾਂ ਦੀ ਸੂਚੀ ਦਿਓ।"
    },
    {
        "prompt": "Write a letter to your favorite author. ਆਪਣੇ ਮਨਪਸੰਦ ਲੇਖਕ ਨੂੰ ਇੱਕ ਚਿੱਠੀ ਲਿਖੋ।"
    },
    {
        "prompt": "Explain the concept of augmented reality. ਵਧੇਰੇ ਹਕੀਕਤ ਦੇ ਅਸੂਲ ਨੂੰ ਸਮਝਾਓ।"
    },
    {
        "prompt": "Translate this text into Arabic. ਇਸ ਪਾਠ ਦਾ ਅਰਬੀ ਵਿੱਚ ਅਨੁਵਾਦ ਕਰੋ।"
    },
    {
        "prompt": "Create a business plan for a new online store. ਇੱਕ ਨਵੇਂ ਆਨਲਾਈਨ ਸਟੋਰ ਲਈ ਵਪਾਰ ਯੋਜਨਾ ਬਣਾਓ।"
    },
    {
        "prompt": "Generate a list of 5 important space missions. 5 ਮਹੱਤਵਪੂਰਨ ਅੰਤਰਿਕਸ਼ ਮਿਸ਼ਨਾਂ ਦੀ ਸੂਚੀ ਬਣਾਓ।"
    },
    {
        "prompt": "Describe the process of learning to swim. ਤੈਰਾਕੀ ਸਿੱਖਣ ਦੀ ਪ੍ਰਕਿਰਿਆ ਦਾ ਵਰਣਨ ਕਰੋ।"
    },
    {
        "prompt": "What are the benefits of joining a sports team? ਇੱਕ ਖੇਡ ਟੀਮ ਵਿੱਚ ਸ਼ਾਮਲ ਹੋਣ ਦੇ ਫਾਇਦੇ ਕੀ ਹਨ?"
    },
    {
        "prompt": "Find the deepest ocean trench in the world. ਦੁਨੀਆ ਦੀ ਸਭ ਤੋਂ ਗਹਿਰੀ ਮਹਾਂਸਾਗਰੀ ਖਾਈ ਲੱਭੋ।"
    },
    {
        "prompt": "How do you create a workout routine? ਤੁਸੀਂ ਇੱਕ ਵਰਕਆਉਟ ਰੂਟੀਨ ਕਿਵੇਂ ਬਣਾਉਂਦੇ ਹੋ?"
    },
    {
        "prompt": "List the top 10 healthiest fruits. ਸਭ ਤੋਂ ਸਿਹਤਮੰਦ 10 ਫਲਾਂ ਦੀ ਸੂਚੀ ਦਿਓ।"
    },
    {
        "prompt": "Write a poem about the sunrise. ਸੂਰਜ ਚੜ੍ਹਦੇ ਬਾਰੇ ਇੱਕ ਕਵਿਤਾ ਲਿਖੋ।"
    },
    {
        "prompt": "Explain the concept of robotics. ਰੋਬੋਟਿਕਸ ਦੇ ਅਸੂਲ ਨੂੰ ਸਮਝਾਓ।"
    },
    {
        "prompt": "Translate this passage into Korean. ਇਸ ਪੈਸੇਜ ਦਾ ਕੋਰੀਅਨ ਵਿੱਚ ਅਨੁਵਾਦ ਕਰੋ।"
    },
    {
        "prompt": "Create a diet plan for weight loss. ਵਜ਼ਨ ਘਟਾਉਣ ਲਈ ਇੱਕ ਖੁਰਾਕ ਯੋਜਨਾ ਬਣਾਓ।"
    },
    {
        "prompt": "Generate a list of 5 famous musicians. 5 ਪ੍ਰਸਿੱਧ ਸੰਗੀਤਕਾਰਾਂ ਦੀ ਸੂਚੀ ਬਣਾਓ।"
    },
    {
        "prompt": "Describe the process of making ice cream. ਆਇਸਕ੍ਰੀਮ ਬਣਾਉਣ ਦੀ ਪ੍ਰਕਿਰਿਆ ਦਾ ਵਰਣਨ ਕਰੋ।"
    },
    {
        "prompt": "What are the benefits of walking daily? ਰੋਜ਼ਾਨਾ ਤੁਰਨ ਦੇ ਫਾਇਦੇ ਕੀ ਹਨ?"
    },
    {
        "prompt": "Find the smallest mammal in the world. ਦੁਨੀਆ ਦਾ ਸਭ ਤੋਂ ਛੋਟਾ ਸਥਣਧਾਰੀ ਲੱਭੋ।"
    },
    {
        "prompt": "How do you write a business email? ਤੁਸੀਂ ਇੱਕ ਵਪਾਰਕ ਈਮੇਲ ਕਿਵੇਂ ਲਿਖਦੇ ਹੋ?"
    },
    {
        "prompt": "List the top 10 best-selling books of all time. ਸਭ ਤੋਂ ਵੱਧ ਵਿਕਣ ਵਾਲੀਆਂ ਕਿਤਾਬਾਂ ਦੀ ਸੂਚੀ ਦਿਓ।"
    }
]


In [10]:
custom_prompts6 = [
    {
        "prompt": "Write a review of your favorite TV show. ਆਪਣੀ ਮਨਪਸੰਦ ਟੀਵੀ ਸ਼ੋ ਦੀ ਸਮੀਖਿਆ ਲਿਖੋ।"
    },
    {
        "prompt": "Explain the concept of machine learning algorithms. ਮਸ਼ੀਨ ਲਰਨਿੰਗ ਅਲਗੋਰਿਦਮ ਦੇ ਅਸੂਲ ਨੂੰ ਸਮਝਾਓ।"
    },
    {
        "prompt": "Translate this paragraph into Portuguese. ਇਸ ਪੈਰਾ ਦਾ ਪੁਰਤਗਾਲੀ ਵਿੱਚ ਅਨੁਵਾਦ ਕਰੋ।"
    },
    {
        "prompt": "Create a summary of a recent news article. ਹਾਲ ਹੀ ਦੀ ਖ਼ਬਰ ਦੇ ਲੇਖ ਦਾ ਇਕ ਸੰਖੇਪ ਬਣਾਓ।"
    },
    {
        "prompt": "Generate a list of 5 benefits of outdoor activities. ਬਾਹਰੀ ਗਤੀਵਿਧੀਆਂ ਦੇ 5 ਫਾਇਦੇ ਦੀ ਸੂਚੀ ਬਣਾਓ।"
    },
    {
        "prompt": "Describe the process of making bread. ਬ੍ਰੇਡ ਬਣਾਉਣ ਦੀ ਪ੍ਰਕਿਰਿਆ ਦਾ ਵਰਣਨ ਕਰੋ।"
    },
    {
        "prompt": "What are the benefits of a ketogenic diet? ਕੈਟੋਜੈਨਿਕ ਖੁਰਾਕ ਦੇ ਫਾਇਦੇ ਕੀ ਹਨ?"
    },
    {
        "prompt": "Find the population of Canada. ਕੈਨੇਡਾ ਦੀ ਅਬਾਦੀ ਲੱਭੋ।"
    },
    {
        "prompt": "How do you solve a linear equation? ਤੁਸੀਂ ਸਿਧਾਰਨ ਸਮੀਕਰਨ ਨੂੰ ਕਿਵੇਂ ਹੱਲ ਕਰਦੇ ਹੋ?"
    },
    {
        "prompt": "List the top 10 tourist attractions in Africa. ਅਫਰੀਕਾ ਵਿੱਚ ਸਿਖਰ ਦੇ 10 ਸੈਲਾਨੀ ਆਕਰਸ਼ਣਾਂ ਦੀ ਸੂਚੀ ਦਿਓ।"
    },
    {
        "prompt": "Write a poem about winter. ਸਰਦੀ ਬਾਰੇ ਇੱਕ ਕਵਿਤਾ ਲਿਖੋ।"
    },
    {
        "prompt": "Explain the concept of cybersecurity. ਸਾਈਬਰ ਸੁਰੱਖਿਆ ਦੇ ਅਸੂਲ ਨੂੰ ਸਮਝਾਓ।"
    },
    {
        "prompt": "Translate this recipe into German. ਇਸ ਰਸੋਈ ਦਾ ਜਰਮਨ ਵਿੱਚ ਅਨੁਵਾਦ ਕਰੋ।"
    },
    {
        "prompt": "Create a travel guide for Sydney. ਸਿਡਨੀ ਲਈ ਇੱਕ ਯਾਤਰਾ ਗਾਈਡ ਬਣਾਓ।"
    },
    {
        "prompt": "Generate a list of 5 programming languages for game development. ਗੇਮ ਵਿਕਾਸ ਲਈ 5 ਪ੍ਰੋਗ੍ਰਾਮਿੰਗ ਭਾਸ਼ਾਵਾਂ ਦੀ ਸੂਚੀ ਬਣਾਓ।"
    },
    {
        "prompt": "Describe the process of setting up a home network. ਇੱਕ ਘਰੇਲੂ ਨੈਟਵਰਕ ਸੈੱਟ ਕਰਨ ਦੀ ਪ੍ਰਕਿਰਿਆ ਦਾ ਵਰਣਨ ਕਰੋ।"
    },
    {
        "prompt": "What are the benefits of journaling? ਜਰਨਲਿੰਗ ਦੇ ਫਾਇਦੇ ਕੀ ਹਨ?"
    },
    {
        "prompt": "Find the highest peak in Europe. ਯੂਰਪ ਦੀ ਸਭ ਤੋਂ ਉੱਚੀ ਚੋਟੀ ਲੱਭੋ।"
    },
    {
        "prompt": "How do you calculate the perimeter of a rectangle? ਤੁਸੀਂ ਆਯਤ ਦੀ ਪਰੇਮੀਟਰ ਕਿਵੇਂ ਗਣਨਾ ਕਰਦੇ ਹੋ?"
    },
    {
        "prompt": "List the top 10 universities in Asia. ਏਸ਼ੀਆ ਦੀਆਂ ਸਿਖਰ ਦੀਆਂ 10 ਯੂਨੀਵਰਸਿਟੀਆਂ ਦੀ ਸੂਚੀ ਦਿਓ।"
    },
    {
        "prompt": "Write a short story about a time traveler. ਇੱਕ ਸਮੇਂ ਦੇ ਯਾਤਰੀ ਬਾਰੇ ਇੱਕ ਛੋਟੀ ਕਹਾਣੀ ਲਿਖੋ।"
    },
    {
        "prompt": "Explain the concept of renewable energy sources. ਨਵੀਨੀਕਰਣਯੋਗ ਊਰਜਾ ਸ੍ਰੋਤਾਂ ਦੇ ਅਸੂਲ ਨੂੰ ਸਮਝਾਓ।"
    },
    {
        "prompt": "Translate this document into Japanese. ਇਸ ਦਸਤਾਵੇਜ਼ ਨੂੰ ਜਪਾਨੀ ਵਿੱਚ ਅਨੁਵਾਦ ਕਰੋ।"
    },
    {
        "prompt": "Create a marketing plan for a new mobile app. ਇੱਕ ਨਵੇਂ ਮੋਬਾਈਲ ਐਪ ਲਈ ਮਾਰਕੀਟਿੰਗ ਯੋਜਨਾ ਬਣਾਓ।"
    },
    {
        "prompt": "Generate a list of 5 famous entrepreneurs. 5 ਪ੍ਰਸਿੱਧ ਉਦਯੋਗਪਤੀਆਂ ਦੀ ਸੂਚੀ ਬਣਾਓ।"
    },
    {
        "prompt": "Describe the process of writing a research paper. ਇੱਕ ਖੋਜ ਪੇਪਰ ਲਿਖਣ ਦੀ ਪ੍ਰਕਿਰਿਆ ਦਾ ਵਰਣਨ ਕਰੋ।"
    },
    {
        "prompt": "What are the benefits of regular exercise? ਨਿਯਮਿਤ ਵਰਜ਼ਿਸ਼ ਦੇ ਫਾਇਦੇ ਕੀ ਹਨ?"
    },
    {
        "prompt": "Find the GDP of Brazil. ਬ੍ਰਾਜ਼ੀਲ ਦਾ GDP ਲੱਭੋ।"
    },
    {
        "prompt": "How do you create a meal plan? ਤੁਸੀਂ ਇੱਕ ਭੋਜਨ ਯੋਜਨਾ ਕਿਵੇਂ ਬਣਾਉਂਦੇ ਹੋ?"
    },
    {
        "prompt": "List the top 10 beaches in the world. ਦੁਨੀਆ ਦੇ ਸਿਖਰ ਦੇ 10 ਸਮੁੰਦਰੀ ਤੱਟਾਂ ਦੀ ਸੂਚੀ ਦਿਓ।"
    },
    {
        "prompt": "Write a letter to your past self. ਆਪਣੇ ਪਿਛਲੇ ਆਪ ਨੂੰ ਇੱਕ ਚਿੱਠੀ ਲਿਖੋ।"
    },
    {
        "prompt": "Explain the concept of artificial intelligence in healthcare. ਸਿਹਤ ਸੰਭਾਲ ਵਿੱਚ ਕ੍ਰਿਤ੍ਰਿਮ ਬੁੱਧੀ ਦੇ ਅਸੂਲ ਨੂੰ ਸਮਝਾਓ।"
    },
    {
        "prompt": "Translate this text into Russian. ਇਸ ਪਾਠ ਦਾ ਰੂਸੀ ਵਿੱਚ ਅਨੁਵਾਦ ਕਰੋ।"
    },
    {
        "prompt": "Create a business plan for a new tech startup. ਇੱਕ ਨਵੇਂ ਤਕਨਾਲੋਜੀ ਸਟਾਰਟਅਪ ਲਈ ਵਪਾਰ ਯੋਜਨਾ ਬਣਾਓ।"
    },
    {
        "prompt": "Generate a list of 5 significant scientific discoveries. 5 ਮਹੱਤਵਪੂਰਨ ਵਿਗਿਆਨਕ ਖੋਜਾਂ ਦੀ ਸੂਚੀ ਬਣਾਓ।"
    },
    {
        "prompt": "Describe the process of learning a new sport. ਇੱਕ ਨਵੀਂ ਖੇਡ ਸਿੱਖਣ ਦੀ ਪ੍ਰਕਿਰਿਆ ਦਾ ਵਰਣਨ ਕਰੋ।"
    },
    {
        "prompt": "What are the benefits of team sports? ਟੀਮ ਖੇਡਾਂ ਦੇ ਫਾਇਦੇ ਕੀ ਹਨ?"
    },
    {
        "prompt": "Find the longest river in Africa. ਅਫਰੀਕਾ ਦੀ ਸਭ ਤੋਂ ਲੰਮੀ ਦਰਿਆ ਲੱਭੋ।"
    },
    {
        "prompt": "How do you create a fitness routine? ਤੁਸੀਂ ਇੱਕ ਫਿਟਨੈਸ ਰੂਟੀਨ ਕਿਵੇਂ ਬਣਾਉਂਦੇ ਹੋ?"
    },
    {
        "prompt": "List the top 10 healthiest vegetables. ਸਭ ਤੋਂ ਸਿਹਤਮੰਦ 10 ਸਬਜ਼ੀਆਂ ਦੀ ਸੂਚੀ ਦਿਓ।"
    },
    {
        "prompt": "Write a poem about the night sky. ਰਾਤ ਦੇ ਅਕਾਸ਼ ਬਾਰੇ ਇੱਕ ਕਵਿਤਾ ਲਿਖੋ।"
    },
    {
        "prompt": "Explain the concept of blockchain technology. ਬਲੌਕਚੇਨ ਤਕਨਾਲੋਜੀ ਦੇ ਅਸੂਲ ਨੂੰ ਸਮਝਾਓ।"
    },
    {
        "prompt": "Translate this passage into Spanish. ਇਸ ਪੈਸੇਜ ਦਾ ਸਪੈਨਿਸ਼ ਵਿੱਚ ਅਨੁਵਾਦ ਕਰੋ।"
    },
    {
        "prompt": "Create a diet plan for muscle gain. ਮਾਸਪੇਸ਼ੀਆਂ ਦੀ ਵਾਧੂ ਲਈ ਇੱਕ ਖੁਰਾਕ ਯੋਜਨਾ ਬਣਾਓ।"
    },
    {
        "prompt": "Generate a list of 5 famous authors. 5 ਪ੍ਰਸਿੱਧ ਲੇਖਕਾਂ ਦੀ ਸੂਚੀ ਬਣਾਓ।"
    },
    {
        "prompt": "Describe the process of making cheese. ਪਨੀਰ ਬਣਾਉਣ ਦੀ ਪ੍ਰਕਿਰਿਆ ਦਾ ਵਰਣਨ ਕਰੋ।"
    },
    {
        "prompt": "What are the benefits of hiking? ਪੈਦਲ ਯਾਤਰਾ ਦੇ ਫਾਇਦੇ ਕੀ ਹਨ?"
    },
    {
        "prompt": "Find the smallest bird in the world. ਦੁਨੀਆ ਦੀ ਸਭ ਤੋਂ ਛੋਟੀ ਪੰਛੀ ਲੱਭੋ।"
    },
    {
        "prompt": "How do you write a resume? ਤੁਸੀਂ ਇੱਕ ਰਿਜ਼ੂਮੇ ਕਿਵੇਂ ਲਿਖਦੇ ਹੋ?"
    },
    {
        "prompt": "List the top 10 movies of the 21st century. 21ਵੇਂ ਸਦੀ ਦੇ ਸਿਖਰ ਦੇ 10 ਫਿਲਮਾਂ ਦੀ ਸੂਚੀ ਦਿਓ।"
    }
]


In [11]:
custom_prompts7 = [
    {
        "prompt": "Write a review of your favorite video game. ਆਪਣੀ ਮਨਪਸੰਦ ਵੀਡੀਓ ਗੇਮ ਦੀ ਸਮੀਖਿਆ ਲਿਖੋ।"
    },
    {
        "prompt": "Explain the concept of virtual machines. ਵਰਚੁਅਲ ਮਸ਼ੀਨਾਂ ਦੇ ਅਸੂਲ ਨੂੰ ਸਮਝਾਓ।"
    },
    {
        "prompt": "Translate this paragraph into Korean. ਇਸ ਪੈਰਾ ਦਾ ਕੋਰੀਅਨ ਵਿੱਚ ਅਨੁਵਾਦ ਕਰੋ।"
    },
    {
        "prompt": "Create a summary of a historical event. ਇੱਕ ਇਤਿਹਾਸਕ ਘਟਨਾ ਦਾ ਇਕ ਸੰਖੇਪ ਬਣਾਓ।"
    },
    {
        "prompt": "Generate a list of 5 benefits of yoga. ਯੋਗ ਦੇ 5 ਫਾਇਦੇ ਦੀ ਸੂਚੀ ਬਣਾਓ।"
    },
    {
        "prompt": "Describe the process of making pancakes. ਪੈਨਕੇਕ ਬਣਾਉਣ ਦੀ ਪ੍ਰਕਿਰਿਆ ਦਾ ਵਰਣਨ ਕਰੋ।"
    },
    {
        "prompt": "What are the benefits of a vegan diet? ਵੀਗਨ ਖੁਰਾਕ ਦੇ ਫਾਇਦੇ ਕੀ ਹਨ?"
    },
    {
        "prompt": "Find the population of Russia. ਰੂਸ ਦੀ ਅਬਾਦੀ ਲੱਭੋ।"
    },
    {
        "prompt": "How do you solve a cubic equation? ਤੁਸੀਂ ਇੱਕ ਘਣ ਸਮੀਕਰਨ ਨੂੰ ਕਿਵੇਂ ਹੱਲ ਕਰਦੇ ਹੋ?"
    },
    {
        "prompt": "List the top 10 tourist attractions in Australia. ਆਸਟ੍ਰੇਲੀਆ ਵਿੱਚ ਸਿਖਰ ਦੇ 10 ਸੈਲਾਨੀ ਆਕਰਸ਼ਣਾਂ ਦੀ ਸੂਚੀ ਦਿਓ।"
    },
    {
        "prompt": "Write a poem about spring. ਬਸੰਤ ਬਾਰੇ ਇੱਕ ਕਵਿਤਾ ਲਿਖੋ।"
    },
    {
        "prompt": "Explain the concept of big data. ਵੱਡੇ ਡੇਟਾ ਦੇ ਅਸੂਲ ਨੂੰ ਸਮਝਾਓ।"
    },
    {
        "prompt": "Translate this recipe into Hindi. ਇਸ ਰਸੋਈ ਦਾ ਹਿੰਦੀ ਵਿੱਚ ਅਨੁਵਾਦ ਕਰੋ।"
    },
    {
        "prompt": "Create a travel guide for Rome. ਰੋਮ ਲਈ ਇੱਕ ਯਾਤਰਾ ਗਾਈਡ ਬਣਾਓ।"
    },
    {
        "prompt": "Generate a list of 5 programming languages for AI development. AI ਵਿਕਾਸ ਲਈ 5 ਪ੍ਰੋਗ੍ਰਾਮਿੰਗ ਭਾਸ਼ਾਵਾਂ ਦੀ ਸੂਚੀ ਬਣਾਓ।"
    },
    {
        "prompt": "Describe the process of setting up a server. ਇੱਕ ਸਰਵਰ ਸੈੱਟ ਕਰਨ ਦੀ ਪ੍ਰਕਿਰਿਆ ਦਾ ਵਰਣਨ ਕਰੋ।"
    },
    {
        "prompt": "What are the benefits of digital detox? ਡਿਜੀਟਲ ਡੀਟੌਕਸ ਦੇ ਫਾਇਦੇ ਕੀ ਹਨ?"
    },
    {
        "prompt": "Find the highest peak in North America. ਉੱਤਰੀ ਅਮਰੀਕਾ ਦੀ ਸਭ ਤੋਂ ਉੱਚੀ ਚੋਟੀ ਲੱਭੋ।"
    },
    {
        "prompt": "How do you calculate the volume of a cone? ਤੁਸੀਂ ਇੱਕ ਕੋਣ ਦਾ ਵਾਲਿਊਮ ਕਿਵੇਂ ਗਣਨਾ ਕਰਦੇ ਹੋ?"
    },
    {
        "prompt": "List the top 10 national parks in the USA. USA ਦੇ ਸਿਖਰ ਦੇ 10 ਰਾਸ਼ਟਰੀ ਪਾਰਕਾਂ ਦੀ ਸੂਚੀ ਦਿਓ।"
    },
    {
        "prompt": "Write a short story about a detective. ਇੱਕ ਜਾਸੂਸ ਬਾਰੇ ਇੱਕ ਛੋਟੀ ਕਹਾਣੀ ਲਿਖੋ।"
    },
    {
        "prompt": "Explain the concept of cloud storage. ਕਲਾਉਡ ਸਟੋਰੇਜ ਦੇ ਅਸੂਲ ਨੂੰ ਸਮਝਾਓ।"
    },
    {
        "prompt": "Translate this document into Italian. ਇਸ ਦਸਤਾਵੇਜ਼ ਨੂੰ ਇਤਾਲਵੀ ਵਿੱਚ ਅਨੁਵਾਦ ਕਰੋ।"
    },
    {
        "prompt": "Create a marketing plan for a new software product. ਇੱਕ ਨਵੇਂ ਸੌਫਟਵੇਅਰ ਉਤਪਾਦ ਲਈ ਮਾਰਕੀਟਿੰਗ ਯੋਜਨਾ ਬਣਾਓ।"
    },
    {
        "prompt": "Generate a list of 5 famous historical battles. 5 ਪ੍ਰਸਿੱਧ ਇਤਿਹਾਸਕ ਲੜਾਈਆਂ ਦੀ ਸੂਚੀ ਬਣਾਓ।"
    },
    {
        "prompt": "Describe the process of writing a novel. ਇੱਕ ਨਾਵਲ ਲਿਖਣ ਦੀ ਪ੍ਰਕਿਰਿਆ ਦਾ ਵਰਣਨ ਕਰੋ।"
    },
    {
        "prompt": "What are the benefits of mindfulness meditation? ਮਾਈੰਡਫੁਲਨੈਸ ਧਿਆਨ ਦੇ ਫਾਇਦੇ ਕੀ ਹਨ?"
    },
    {
        "prompt": "Find the GDP of Mexico. ਮੈਕਸੀਕੋ ਦਾ GDP ਲੱਭੋ।"
    },
    {
        "prompt": "How do you create a savings plan? ਤੁਸੀਂ ਇੱਕ ਬਚਤ ਯੋਜਨਾ ਕਿਵੇਂ ਬਣਾਉਂਦੇ ਹੋ?"
    },
    {
        "prompt": "List the top 10 honeymoon destinations. ਹਨੀਮੂਨ ਲਈ ਸਿਖਰ ਦੇ 10 ਮੰਜ਼ਿਲਾਂ ਦੀ ਸੂਚੀ ਦਿਓ।"
    },
    {
        "prompt": "Write a letter to a historical figure. ਇੱਕ ਇਤਿਹਾਸਕ ਹਸਤੀ ਨੂੰ ਇੱਕ ਚਿੱਠੀ ਲਿਖੋ।"
    },
    {
        "prompt": "Explain the concept of deep learning. ਡੀਪ ਲਰਨਿੰਗ ਦੇ ਅਸੂਲ ਨੂੰ ਸਮਝਾਓ।"
    },
    {
        "prompt": "Translate this text into French. ਇਸ ਪਾਠ ਦਾ ਫਰੈਂਚ ਵਿੱਚ ਅਨੁਵਾਦ ਕਰੋ।"
    },
    {
        "prompt": "Create a business plan for a new bakery. ਇੱਕ ਨਵੀਂ ਬੇਕਰੀ ਲਈ ਵਪਾਰ ਯੋਜਨਾ ਬਣਾਓ।"
    },
    {
        "prompt": "Generate a list of 5 groundbreaking scientific theories. 5 ਪ੍ਰਗਟਿਆਸ਼ੀਲ ਵਿਗਿਆਨਕ ਸਿਧਾਂਤਾਂ ਦੀ ਸੂਚੀ ਬਣਾਓ।"
    },
    {
        "prompt": "Describe the process of learning to paint. ਪੇਂਟਿੰਗ ਸਿੱਖਣ ਦੀ ਪ੍ਰਕਿਰਿਆ ਦਾ ਵਰਣਨ ਕਰੋ।"
    },
    {
        "prompt": "What are the benefits of reading fiction? ਕਲਪਨਾਤਮਕ ਕਿਤਾਬਾਂ ਪੜ੍ਹਨ ਦੇ ਫਾਇਦੇ ਕੀ ਹਨ?"
    },
    {
        "prompt": "Find the largest lake in the world. ਦੁਨੀਆ ਦੀ ਸਭ ਤੋਂ ਵੱਡੀ ਝੀਲ ਲੱਭੋ।"
    },
    {
        "prompt": "How do you create a content strategy? ਤੁਸੀਂ ਇੱਕ ਸਮੱਗਰੀ ਰਣਨੀਤੀ ਕਿਵੇਂ ਬਣਾਉਂਦੇ ਹੋ?"
    },
    {
        "prompt": "List the top 10 highest-grossing films of all time. ਸਭ ਤੋਂ ਵੱਧ ਕਮਾਈ ਕਰਨ ਵਾਲੀਆਂ 10 ਫਿਲਮਾਂ ਦੀ ਸੂਚੀ ਦਿਓ।"
    },
    {
        "prompt": "Write a poem about the ocean. ਮਹਾਂਸਾਗਰ ਬਾਰੇ ਇੱਕ ਕਵਿਤਾ ਲਿਖੋ।"
    },
    {
        "prompt": "Explain the concept of machine translation. ਮਸ਼ੀਨ ਅਨੁਵਾਦ ਦੇ ਅਸੂਲ ਨੂੰ ਸਮਝਾਓ।"
    },
    {
        "prompt": "Translate this passage into Portuguese. ਇਸ ਪੈਸੇਜ ਦਾ ਪੁਰਤਗਾਲੀ ਵਿੱਚ ਅਨੁਵਾਦ ਕਰੋ।"
    },
    {
        "prompt": "Create a diet plan for athletes. ਖਿਡਾਰੀਆਂ ਲਈ ਇੱਕ ਖੁਰਾਕ ਯੋਜਨਾ ਬਣਾਓ।"
    },
    {
        "prompt": "Generate a list of 5 influential political leaders. 5 ਪ੍ਰਭਾਵਸ਼ਾਲੀ ਰਾਜਨੀਤਿਕ ਆਗੂਆਂ ਦੀ ਸੂਚੀ ਬਣਾਓ।"
    },
    {
        "prompt": "Describe the process of making soap. ਸਾਬਣ ਬਣਾਉਣ ਦੀ ਪ੍ਰਕਿਰਿਆ ਦਾ ਵਰਣਨ ਕਰੋ।"
    },
    {
        "prompt": "What are the benefits of volunteering? ਸੇਵਾ ਕਰਨ ਦੇ ਫਾਇਦੇ ਕੀ ਹਨ?"
    },
    {
        "prompt": "Find the smallest country in Europe. ਯੂਰਪ ਦਾ ਸਭ ਤੋਂ ਛੋਟਾ ਦੇਸ਼ ਲੱਭੋ।"
    },
    {
        "prompt": "How do you write a cover letter? ਤੁਸੀਂ ਇੱਕ ਕਵਰ ਲੈਟਰ ਕਿਵੇਂ ਲਿਖਦੇ ਹੋ?"
    },
    {
        "prompt": "List the top 10 best-selling albums of all time. ਸਭ ਤੋਂ ਵੱਧ ਵਿਕਣ ਵਾਲੇ 10 ਐਲਬਮਾਂ ਦੀ ਸੂਚੀ ਦਿਓ।"
    }
]

In [12]:
custom_prompts8 = [
    {
        "prompt": "Write a review of your favorite album. ਆਪਣੀ ਮਨਪਸੰਦ ਐਲਬਮ ਦੀ ਸਮੀਖਿਆ ਲਿਖੋ।"
    },
    {
        "prompt": "Explain the concept of quantum physics. ਕਵਾਂਟਮ ਭੌਤਿਕ ਵਿਦਿਆ ਦੇ ਅਸੂਲ ਨੂੰ ਸਮਝਾਓ।"
    },
    {
        "prompt": "Translate this paragraph into Swahili. ਇਸ ਪੈਰਾ ਦਾ ਸਵਾਹਿਲੀ ਵਿੱਚ ਅਨੁਵਾਦ ਕਰੋ।"
    },
    {
        "prompt": "Create a summary of a scientific article. ਇੱਕ ਵਿਗਿਆਨਕ ਲੇਖ ਦਾ ਇਕ ਸੰਖੇਪ ਬਣਾਓ।"
    },
    {
        "prompt": "Generate a list of 5 benefits of forest conservation. ਜੰਗਲ ਸੰਰਖਣ ਦੇ 5 ਫਾਇਦੇ ਦੀ ਸੂਚੀ ਬਣਾਓ।"
    },
    {
        "prompt": "Describe the process of making pottery. ਕੂੰਭਾਰ ਦਾ ਕੰਮ ਬਣਾਉਣ ਦੀ ਪ੍ਰਕਿਰਿਆ ਦਾ ਵਰਣਨ ਕਰੋ।"
    },
    {
        "prompt": "What are the benefits of intermittent fasting? ਅਨਿਯਮਿਤ ਉਪਵਾਸ ਦੇ ਫਾਇਦੇ ਕੀ ਹਨ?"
    },
    {
        "prompt": "Find the population of Nigeria. ਨਾਈਜੀਰੀਆ ਦੀ ਅਬਾਦੀ ਲੱਭੋ।"
    },
    {
        "prompt": "How do you solve a differential equation? ਤੁਸੀਂ ਇੱਕ ਡਿਫ਼ਰੈਂਸ਼ਲ ਸਮੀਕਰਨ ਨੂੰ ਕਿਵੇਂ ਹੱਲ ਕਰਦੇ ਹੋ?"
    },
    {
        "prompt": "List the top 10 tourist attractions in South America. ਦੱਖਣੀ ਅਮਰੀਕਾ ਵਿੱਚ ਸਿਖਰ ਦੇ 10 ਸੈਲਾਨੀ ਆਕਰਸ਼ਣਾਂ ਦੀ ਸੂਚੀ ਦਿਓ।"
    },
    {
        "prompt": "Write a poem about the desert. ਰੇਗਿਸਤਾਨ ਬਾਰੇ ਇੱਕ ਕਵਿਤਾ ਲਿਖੋ।"
    },
    {
        "prompt": "Explain the concept of renewable energy storage. ਨਵੀਨੀਕਰਣ ਯੋਗ ਊਰਜਾ ਸਟੋਰੇਜ ਦੇ ਅਸੂਲ ਨੂੰ ਸਮਝਾਓ।"
    },
    {
        "prompt": "Translate this recipe into Greek. ਇਸ ਰਸੋਈ ਦਾ ਗ੍ਰੀਕ ਵਿੱਚ ਅਨੁਵਾਦ ਕਰੋ।"
    },
    {
        "prompt": "Create a travel guide for Istanbul. ਇਸਤਾਂਬੁਲ ਲਈ ਇੱਕ ਯਾਤਰਾ ਗਾਈਡ ਬਣਾਓ।"
    },
    {
        "prompt": "Generate a list of 5 programming languages for robotics. ਰੋਬੋਟਿਕਸ ਲਈ 5 ਪ੍ਰੋਗ੍ਰਾਮਿੰਗ ਭਾਸ਼ਾਵਾਂ ਦੀ ਸੂਚੀ ਬਣਾਓ।"
    },
    {
        "prompt": "Describe the process of installing solar panels. ਸੂਰਜੀ ਪੈਨਲ ਲਗਾਉਣ ਦੀ ਪ੍ਰਕਿਰਿਆ ਦਾ ਵਰਣਨ ਕਰੋ।"
    },
    {
        "prompt": "What are the benefits of sustainable farming? ਸਥਿਰ ਖੇਤੀਬਾੜੀ ਦੇ ਫਾਇਦੇ ਕੀ ਹਨ?"
    },
    {
        "prompt": "Find the highest peak in Antarctica. ਅੰਟਾਰਕਟਿਕਾ ਦੀ ਸਭ ਤੋਂ ਉੱਚੀ ਚੋਟੀ ਲੱਭੋ।"
    },
    {
        "prompt": "How do you calculate the area of a parallelogram? ਤੁਸੀਂ ਇੱਕ ਸਾਮਾਂਤਰ ਚਤੁਰਭੁਜ ਦਾ ਖੇਤਰਫਲ ਕਿਵੇਂ ਗਣਨਾ ਕਰਦੇ ਹੋ?"
    },
    {
        "prompt": "List the top 10 ancient civilizations. ਪ੍ਰਾਚੀਨ ਸਭਿਆਚਾਰਾਂ ਦੀ ਸਿਖਰ ਦੇ 10 ਸੂਚੀ ਦਿਓ।"
    },
    {
        "prompt": "Write a short story about a mythical creature. ਇੱਕ ਦੰਪਤ ਕ੍ਰੀਚਰ ਬਾਰੇ ਇੱਕ ਛੋਟੀ ਕਹਾਣੀ ਲਿਖੋ।"
    },
    {
        "prompt": "Explain the concept of genetic modification. ਜੈਨੇਟਿਕ ਸੰਸ਼ੋਧਨ ਦੇ ਅਸੂਲ ਨੂੰ ਸਮਝਾਓ।"
    },
    {
        "prompt": "Translate this document into Dutch. ਇਸ ਦਸਤਾਵੇਜ਼ ਨੂੰ ਡੱਚ ਵਿੱਚ ਅਨੁਵਾਦ ਕਰੋ।"
    },
    {
        "prompt": "Create a marketing plan for a new fitness brand. ਇੱਕ ਨਵੇਂ ਫਿਟਨੈਸ ਬ੍ਰਾਂਡ ਲਈ ਮਾਰਕੀਟਿੰਗ ਯੋਜਨਾ ਬਣਾਓ।"
    },
    {
        "prompt": "Generate a list of 5 notable space missions. 5 ਪ੍ਰਸਿੱਧ ਅੰਤਰਿਕਸ਼ ਮਿਸ਼ਨਾਂ ਦੀ ਸੂਚੀ ਬਣਾਓ।"
    },
    {
        "prompt": "Describe the process of writing a grant proposal. ਇੱਕ ਅਨੁਦਾਨ ਪ੍ਰਸਤਾਵ ਲਿਖਣ ਦੀ ਪ੍ਰਕਿਰਿਆ ਦਾ ਵਰਣਨ ਕਰੋ।"
    },
    {
        "prompt": "What are the benefits of cross-cultural exchange? ਅੰਤਰ-ਸੱਭਿਆਚਾਰਕ ਤਬਾਦਲੇ ਦੇ ਫਾਇਦੇ ਕੀ ਹਨ?"
    },
    {
        "prompt": "Find the GDP of South Korea. ਦੱਖਣੀ ਕੋਰੀਆ ਦਾ GDP ਲੱਭੋ।"
    },
    {
        "prompt": "How do you create a community outreach program? ਤੁਸੀਂ ਇੱਕ ਕਮਿਊਨਿਟੀ ਆਉਟਰੀਚ ਪ੍ਰੋਗਰਾਮ ਕਿਵੇਂ ਬਣਾਉਂਦੇ ਹੋ?"
    },
    {
        "prompt": "List the top 10 most visited cities in the world. ਦੁਨੀਆ ਦੇ ਸਭ ਤੋਂ ਵੱਧ ਵਿਜ਼ਿਟ ਕੀਤੇ ਗਏ 10 ਸ਼ਹਿਰਾਂ ਦੀ ਸੂਚੀ ਦਿਓ।"
    },
    {
        "prompt": "Write a letter to a future world leader. ਇੱਕ ਭਵਿੱਖ ਦੇ ਵਿਸ਼ਵ ਆਗੂ ਨੂੰ ਇੱਕ ਚਿੱਠੀ ਲਿਖੋ।"
    },
    {
        "prompt": "Explain the concept of augmented reality. ਵਧੇਰੇ ਹਕੀਕਤ ਦੇ ਅਸੂਲ ਨੂੰ ਸਮਝਾਓ।"
    },
    {
        "prompt": "Translate this text into Arabic. ਇਸ ਪਾਠ ਦਾ ਅਰਬੀ ਵਿੱਚ ਅਨੁਵਾਦ ਕਰੋ।"
    },
    {
        "prompt": "Create a business plan for an eco-friendly product. ਇੱਕ ਪਰੀਆਵਰਨ-ਮਿੱਤਰ ਉਤਪਾਦ ਲਈ ਵਪਾਰ ਯੋਜਨਾ ਬਣਾਓ।"
    },
    {
        "prompt": "Generate a list of 5 significant historical documents. 5 ਮਹੱਤਵਪੂਰਨ ਇਤਿਹਾਸਕ ਦਸਤਾਵੇਜ਼ਾਂ ਦੀ ਸੂਚੀ ਬਣਾਓ।"
    },
    {
        "prompt": "Describe the process of learning to dance. ਨੱਚਣਾ ਸਿੱਖਣ ਦੀ ਪ੍ਰਕਿਰਿਆ ਦਾ ਵਰਣਨ ਕਰੋ।"
    },
    {
        "prompt": "What are the benefits of public transportation? ਜਨਤਕ ਆਵਾਜਾਈ ਦੇ ਫਾਇਦੇ ਕੀ ਹਨ?"
    },
    {
        "prompt": "Find the longest coastline in the world. ਦੁਨੀਆ ਦੀ ਸਭ ਤੋਂ ਲੰਮੀ ਤਟਰੇਖਾ ਲੱਭੋ।"
    },
    {
        "prompt": "How do you create a social media strategy? ਤੁਸੀਂ ਇੱਕ ਸੋਸ਼ਲ ਮੀਡੀਆ ਰਣਨੀਤੀ ਕਿਵੇਂ ਬਣਾਉਂਦੇ ਹੋ?"
    },
    {
        "prompt": "List the top 10 influential women in history. ਇਤਿਹਾਸ ਵਿੱਚ 10 ਪ੍ਰਭਾਵਸ਼ਾਲੀ ਔਰਤਾਂ ਦੀ ਸੂਚੀ ਦਿਓ।"
    },
    {
        "prompt": "Write a poem about friendship. ਦੋਸਤੀ ਬਾਰੇ ਇੱਕ ਕਵਿਤਾ ਲਿਖੋ।"
    },
    {
        "prompt": "Explain the concept of telemedicine. ਟੈਲੀਮੈਡਿਸਨ ਦੇ ਅਸੂਲ ਨੂੰ ਸਮਝਾਓ।"
    },
    {
        "prompt": "Translate this passage into Russian. ਇਸ ਪੈਸੇਜ ਦਾ ਰੂਸੀ ਵਿੱਚ ਅਨੁਵਾਦ ਕਰੋ।"
    },
    {
        "prompt": "Create a diet plan for a diabetic person. ਇੱਕ ਮਧੁਮੇਹ ਰੋਗੀ ਲਈ ਇੱਕ ਖੁਰਾਕ ਯੋਜਨਾ ਬਣਾਓ।"
    },
    {
        "prompt": "Generate a list of 5 notable inventions of the 21st century. 21ਵੀਂ ਸਦੀ ਦੀਆਂ 5 ਪ੍ਰਸਿੱਧ ਖੋਜਾਂ ਦੀ ਸੂਚੀ ਬਣਾਓ।"
    },
    {
        "prompt": "Describe the process of making candles. ਮੋਮਬੱਤੀਆਂ ਬਣਾਉਣ ਦੀ ਪ੍ਰਕਿਰਿਆ ਦਾ ਵਰਣਨ ਕਰੋ।"
    },
    {
        "prompt": "What are the benefits of learning a musical instrument? ਇੱਕ ਸੰਗੀਤਕ ਸਾਜ਼ ਸਿੱਖਣ ਦੇ ਫਾਇਦੇ ਕੀ ਹਨ?"
    },
    {
        "prompt": "Find the largest coral reef in the world. ਦੁਨੀਆ ਦੀ ਸਭ ਤੋਂ ਵੱਡੀ ਕੋਰਲ ਰੀਫ਼ ਲੱਭੋ।"
    },
    {
        "prompt": "How do you write an academic paper? ਤੁਸੀਂ ਇੱਕ ਅਕਾਦਮਿਕ ਪੇਪਰ ਕਿਵੇਂ ਲਿਖਦੇ ਹੋ?"
    },
    {
        "prompt": "List the top 10 fashion designers of all time. ਸਭ ਤੋਂ ਵੱਧ ਪ੍ਰਸਿੱਧ 10 ਫੈਸ਼ਨ ਡਿਜ਼ਾਈਨਰਾਂ ਦੀ ਸੂਚੀ ਦਿਓ।"
    }
]


In [13]:
custom_prompts9 = [
    {
        "prompt": "Write a review of your favorite coffee shop. ਆਪਣੀ ਮਨਪਸੰਦ ਕਾਫੀ ਦੁਕਾਨ ਦੀ ਸਮੀਖਿਆ ਲਿਖੋ।"
    },
    {
        "prompt": "Explain the concept of blockchain technology. ਬਲੌਕਚੇਨ ਤਕਨਾਲੋਜੀ ਦੇ ਅਸੂਲ ਨੂੰ ਸਮਝਾਓ।"
    },
    {
        "prompt": "Translate this paragraph into Arabic. ਇਸ ਪੈਰਾ ਦਾ ਅਰਬੀ ਵਿੱਚ ਅਨੁਵਾਦ ਕਰੋ।"
    },
    {
        "prompt": "Create a summary of a fantasy novel. ਇੱਕ ਫੈਂਟਸੀ ਨਾਵਲ ਦਾ ਇਕ ਸੰਖੇਪ ਬਣਾਓ।"
    },
    {
        "prompt": "Generate a list of 5 benefits of urban gardening. ਸ਼ਹਿਰੀ ਬਾਗਬਾਨੀ ਦੇ 5 ਫਾਇਦੇ ਦੀ ਸੂਚੀ ਬਣਾਓ।"
    },
    {
        "prompt": "Describe the process of making sourdough bread. ਸੋਰਡੌ ਬ੍ਰੇਡ ਬਣਾਉਣ ਦੀ ਪ੍ਰਕਿਰਿਆ ਦਾ ਵਰਣਨ ਕਰੋ।"
    },
    {
        "prompt": "What are the benefits of a high-fiber diet? ਉੱਚ ਫਾਈਬਰ ਖੁਰਾਕ ਦੇ ਫਾਇਦੇ ਕੀ ਹਨ?"
    },
    {
        "prompt": "Find the population of Indonesia. ਇੰਡੋਨੇਸ਼ੀਆ ਦੀ ਅਬਾਦੀ ਲੱਭੋ।"
    },
    {
        "prompt": "How do you solve a simultaneous equation? ਤੁਸੀਂ ਇੱਕ ਸਮਾਂਤਰ ਸਮੀਕਰਨ ਨੂੰ ਕਿਵੇਂ ਹੱਲ ਕਰਦੇ ਹੋ?"
    },
    {
        "prompt": "List the top 10 festivals celebrated worldwide. ਦੁਨੀਆ ਭਰ ਵਿੱਚ ਮਨਾਏ ਜਾਣ ਵਾਲੇ ਸਿਖਰ ਦੇ 10 ਤਿਉਹਾਰਾਂ ਦੀ ਸੂਚੀ ਦਿਓ।"
    },
    {
        "prompt": "Write a poem about a mountain hike. ਇੱਕ ਪਹਾੜੀ ਚੜਾਈ ਬਾਰੇ ਇੱਕ ਕਵਿਤਾ ਲਿਖੋ।"
    },
    {
        "prompt": "Explain the concept of green energy. ਗ੍ਰੀਨ ਊਰਜਾ ਦੇ ਅਸੂਲ ਨੂੰ ਸਮਝਾਓ।"
    },
    {
        "prompt": "Translate this recipe into Thai. ਇਸ ਰਸੋਈ ਦਾ ਥਾਈ ਵਿੱਚ ਅਨੁਵਾਦ ਕਰੋ।"
    },
    {
        "prompt": "Create a travel guide for Barcelona. ਬਾਰਸੇਲੋਨਾ ਲਈ ਇੱਕ ਯਾਤਰਾ ਗਾਈਡ ਬਣਾਓ।"
    },
    {
        "prompt": "Generate a list of 5 programming languages for data science. ਡਾਟਾ ਸਾਇੰਸ ਲਈ 5 ਪ੍ਰੋਗ੍ਰਾਮਿੰਗ ਭਾਸ਼ਾਵਾਂ ਦੀ ਸੂਚੀ ਬਣਾਓ।"
    },
    {
        "prompt": "Describe the process of setting up a hydroponic garden. ਇੱਕ ਹਾਈਡ੍ਰੋਪੋਨਿਕ ਬਾਗਬਾਨੀ ਸੈੱਟ ਕਰਨ ਦੀ ਪ੍ਰਕਿਰਿਆ ਦਾ ਵਰਣਨ ਕਰੋ।"
    },
    {
        "prompt": "What are the benefits of digital payments? ਡਿਜੀਟਲ ਭੁਗਤਾਨ ਦੇ ਫਾਇਦੇ ਕੀ ਹਨ?"
    },
    {
        "prompt": "Find the highest peak in Africa. ਅਫਰੀਕਾ ਦੀ ਸਭ ਤੋਂ ਉੱਚੀ ਚੋਟੀ ਲੱਭੋ।"
    },
    {
        "prompt": "How do you calculate the volume of a pyramid? ਤੁਸੀਂ ਇੱਕ ਪਿਰਾਮਿਡ ਦਾ ਵਾਲਿਊਮ ਕਿਵੇਂ ਗਣਨਾ ਕਰਦੇ ਹੋ?"
    },
    {
        "prompt": "List the top 10 endangered species. ਖਤਰੇ ਵਿੱਚ ਪੈਣ ਵਾਲੀਆਂ 10 ਪ੍ਰਜਾਤੀਆਂ ਦੀ ਸੂਚੀ ਦਿਓ।"
    },
    {
        "prompt": "Write a short story about a talking tree. ਇੱਕ ਬੋਲਦੇ ਰੁੱਖ ਬਾਰੇ ਇੱਕ ਛੋਟੀ ਕਹਾਣੀ ਲਿਖੋ।"
    },
    {
        "prompt": "Explain the concept of carbon footprint. ਕਾਰਬਨ ਫੁਟਪ੍ਰਿੰਟ ਦੇ ਅਸੂਲ ਨੂੰ ਸਮਝਾਓ।"
    },
    {
        "prompt": "Translate this document into Swedish. ਇਸ ਦਸਤਾਵੇਜ਼ ਨੂੰ ਸਵੀਡਿਸ਼ ਵਿੱਚ ਅਨੁਵਾਦ ਕਰੋ।"
    },
    {
        "prompt": "Create a marketing plan for a new electric vehicle. ਇੱਕ ਨਵੇਂ ਬਿਜਲੀ ਵਾਲੇ ਵਾਹਨ ਲਈ ਮਾਰਕੀਟਿੰਗ ਯੋਜਨਾ ਬਣਾਓ।"
    },
    {
        "prompt": "Generate a list of 5 famous art movements. 5 ਪ੍ਰਸਿੱਧ ਕਲਾ ਅੰਦੋਲਨਾਂ ਦੀ ਸੂਚੀ ਬਣਾਓ।"
    },
    {
        "prompt": "Describe the process of writing a screenplay. ਇੱਕ ਸਕ੍ਰੀਨਪਲੇ ਲਿਖਣ ਦੀ ਪ੍ਰਕਿਰਿਆ ਦਾ ਵਰਣਨ ਕਰੋ।"
    },
    {
        "prompt": "What are the benefits of plant-based diets? ਪੌਧੇ-ਅਧਾਰਿਤ ਖੁਰਾਕ ਦੇ ਫਾਇਦੇ ਕੀ ਹਨ?"
    },
    {
        "prompt": "Find the GDP of Argentina. ਅਰਜਨਟੀਨਾ ਦਾ GDP ਲੱਭੋ।"
    },
    {
        "prompt": "How do you create a fitness plan? ਤੁਸੀਂ ਇੱਕ ਫਿਟਨੈਸ ਯੋਜਨਾ ਕਿਵੇਂ ਬਣਾਉਂਦੇ ਹੋ?"
    },
    {
        "prompt": "List the top 10 most populated cities. ਸਭ ਤੋਂ ਵੱਧ ਅਬਾਦੀ ਵਾਲੇ 10 ਸ਼ਹਿਰਾਂ ਦੀ ਸੂਚੀ ਦਿਓ।"
    },
    {
        "prompt": "Write a letter to your childhood hero. ਆਪਣੇ ਬਚਪਨ ਦੇ ਹੀਰੋ ਨੂੰ ਇੱਕ ਚਿੱਠੀ ਲਿਖੋ।"
    },
    {
        "prompt": "Explain the concept of augmented reality in gaming. ਗੇਮਿੰਗ ਵਿੱਚ ਵਧੀਕ ਹਕੀਕਤ ਦੇ ਅਸੂਲ ਨੂੰ ਸਮਝਾਓ।"
    },
    {
        "prompt": "Translate this text into Hebrew. ਇਸ ਪਾਠ ਦਾ ਹੀਬਰੂ ਵਿੱਚ ਅਨੁਵਾਦ ਕਰੋ।"
    },
    {
        "prompt": "Create a business plan for a renewable energy company. ਇੱਕ ਨਵੀਨੀਕਰਣਯੋਗ ਊਰਜਾ ਕੰਪਨੀ ਲਈ ਵਪਾਰ ਯੋਜਨਾ ਬਣਾਓ।"
    },
    {
        "prompt": "Generate a list of 5 landmark Supreme Court cases. 5 ਮਹੱਤਵਪੂਰਨ ਸੁਪਰੀਮ ਕੋਰਟ ਦੇ ਮਾਮਲਿਆਂ ਦੀ ਸੂਚੀ ਬਣਾਓ।"
    },
    {
        "prompt": "Describe the process of learning a new language. ਇੱਕ ਨਵੀਂ ਭਾਸ਼ਾ ਸਿੱਖਣ ਦੀ ਪ੍ਰਕਿਰਿਆ ਦਾ ਵਰਣਨ ਕਰੋ।"
    },
    {
        "prompt": "What are the benefits of mindfulness practices? ਮਾਈੰਡਫੁਲਨੈਸ ਅਭਿਆਸ ਦੇ ਫਾਇਦੇ ਕੀ ਹਨ?"
    },
    {
        "prompt": "Find the deepest lake in the world. ਦੁਨੀਆ ਦੀ ਸਭ ਤੋਂ ਗਹਿਰੀ ਝੀਲ ਲੱਭੋ।"
    },
    {
        "prompt": "How do you create an effective study schedule? ਤੁਸੀਂ ਇੱਕ ਪ੍ਰਭਾਵਸ਼ਾਲੀ ਅਧਿਐਨ ਸੂਚੀ ਕਿਵੇਂ ਬਣਾਉਂਦੇ ਹੋ?"
    },
    {
        "prompt": "List the top 10 most innovative companies. ਸਭ ਤੋਂ ਨਵੀਨਤਮ 10 ਕੰਪਨੀਆਂ ਦੀ ਸੂਚੀ ਦਿਓ।"
    },
    {
        "prompt": "Write a poem about the night sky. ਰਾਤ ਦੇ ਅਕਾਸ਼ ਬਾਰੇ ਇੱਕ ਕਵਿਤਾ ਲਿਖੋ।"
    },
    {
        "prompt": "Explain the concept of gene editing. ਜਿਨੋਮ ਸੰਪਾਦਨ ਦੇ ਅਸੂਲ ਨੂੰ ਸਮਝਾਓ।"
    },
    {
        "prompt": "Translate this passage into Vietnamese. ਇਸ ਪੈਸੇਜ ਦਾ ਵਿੜਨਾਮੀ ਵਿੱਚ ਅਨੁਵਾਦ ਕਰੋ।"
    },
    {
        "prompt": "Create a diet plan for weight gain. ਵਜ਼ਨ ਵਧਾਉਣ ਲਈ ਇੱਕ ਖੁਰਾਕ ਯੋਜਨਾ ਬਣਾਓ।"
    },
    {
        "prompt": "Generate a list of 5 influential music composers. 5 ਪ੍ਰਭਾਵਸ਼ਾਲੀ ਸੰਗੀਤ ਰਚਨਾਕਾਰਾਂ ਦੀ ਸੂਚੀ ਬਣਾਓ।"
    },
    {
        "prompt": "Describe the process of making herbal tea. ਜੜੀ-ਬੂਟੀਆਂ ਵਾਲੀ ਚਾਹ ਬਣਾਉਣ ਦੀ ਪ੍ਰਕਿਰਿਆ ਦਾ ਵਰਣਨ ਕਰੋ।"
    },
    {
        "prompt": "What are the benefits of swimming? ਤੈਰਾਕੀ ਦੇ ਫਾਇਦੇ ਕੀ ਹਨ?"
    },
    {
        "prompt": "Find the smallest mammal in the world. ਦੁਨੀਆ ਦਾ ਸਭ ਤੋਂ ਛੋਟਾ ਸਥਣਧਾਰੀ ਲੱਭੋ।"
    },
    {
        "prompt": "How do you write a technical report? ਤੁਸੀਂ ਇੱਕ ਤਕਨੀਕੀ ਰਿਪੋਰਟ ਕਿਵੇਂ ਲਿਖਦੇ ਹੋ?"
    },
    {
        "prompt": "List the top 10 most valuable brands in the world. ਦੁਨੀਆ ਦੇ ਸਭ ਤੋਂ ਕੀਮਤੀ 10 ਬ੍ਰਾਂਡਾਂ ਦੀ ਸੂਚੀ ਦਿਓ।"
    }
]


In [14]:
custom_prompts10 = [
    {
        "prompt": "Write a review of your favorite TV series. ਆਪਣੀ ਮਨਪਸੰਦ ਟੀਵੀ ਸੀਰੀਜ਼ ਦੀ ਸਮੀਖਿਆ ਲਿਖੋ।"
    },
    {
        "prompt": "Explain the concept of machine learning in finance. ਵਿੱਤੀ ਮਸ਼ੀਨ ਲਰਨਿੰਗ ਦੇ ਅਸੂਲ ਨੂੰ ਸਮਝਾਓ।"
    },
    {
        "prompt": "Translate this paragraph into German. ਇਸ ਪੈਰਾ ਦਾ ਜਰਮਨ ਵਿੱਚ ਅਨੁਵਾਦ ਕਰੋ।"
    },
    {
        "prompt": "Create a summary of a documentary you watched. ਤੁਸੀਂ ਦੇਖੀ ਇਕ ਡਾਕੂਮੈਂਟਰੀ ਦਾ ਸੰਖੇਪ ਬਣਾਓ।"
    },
    {
        "prompt": "Generate a list of 5 benefits of biodiversity. ਜੈਵ ਵਿਭਿੰਨਤਾ ਦੇ 5 ਫਾਇਦੇ ਦੀ ਸੂਚੀ ਬਣਾਓ।"
    },
    {
        "prompt": "Describe the process of making kombucha. ਕੋਮਬੁਚਾ ਬਣਾਉਣ ਦੀ ਪ੍ਰਕਿਰਿਆ ਦਾ ਵਰਣਨ ਕਰੋ।"
    },
    {
        "prompt": "What are the benefits of eating organic food? ਜੈਵਿਕ ਖਾਣਾ ਖਾਣ ਦੇ ਫਾਇਦੇ ਕੀ ਹਨ?"
    },
    {
        "prompt": "Find the population of Egypt. ਮਿਸਰ ਦੀ ਅਬਾਦੀ ਲੱਭੋ।"
    },
    {
        "prompt": "How do you solve a quadratic equation using factoring? ਤੁਸੀਂ ਫੈਕਟਰੀਜ਼ੇਸ਼ਨ ਦੀ ਵਰਤੋਂ ਕਰਕੇ ਇੱਕ ਚਤੁਰਭੁਜ ਸਮੀਕਰਨ ਕਿਵੇਂ ਹੱਲ ਕਰਦੇ ਹੋ?"
    },
    {
        "prompt": "List the top 10 national parks in Africa. ਅਫਰੀਕਾ ਦੇ ਸਿਖਰ ਦੇ 10 ਰਾਸ਼ਟਰੀ ਪਾਰਕਾਂ ਦੀ ਸੂਚੀ ਦਿਓ।"
    },
    {
        "prompt": "Write a poem about the ocean waves. ਮਹਾਂਸਾਗਰੀ ਲਹਿਰਾਂ ਬਾਰੇ ਇੱਕ ਕਵਿਤਾ ਲਿਖੋ।"
    },
    {
        "prompt": "Explain the concept of circular economy. ਗੋਲਾਕਾਰ ਅਰਥਵਿਵਸਥਾ ਦੇ ਅਸੂਲ ਨੂੰ ਸਮਝਾਓ।"
    },
    {
        "prompt": "Translate this recipe into Russian. ਇਸ ਰਸੋਈ ਦਾ ਰੂਸੀ ਵਿੱਚ ਅਨੁਵਾਦ ਕਰੋ।"
    },
    {
        "prompt": "Create a travel guide for Paris. ਪੈਰਿਸ ਲਈ ਇੱਕ ਯਾਤਰਾ ਗਾਈਡ ਬਣਾਓ।"
    },
    {
        "prompt": "Generate a list of 5 programming languages for blockchain development. ਬਲੌਕਚੇਨ ਵਿਕਾਸ ਲਈ 5 ਪ੍ਰੋਗ੍ਰਾਮਿੰਗ ਭਾਸ਼ਾਵਾਂ ਦੀ ਸੂਚੀ ਬਣਾਓ।"
    },
    {
        "prompt": "Describe the process of setting up a smart home system. ਇੱਕ ਸਮਾਰਟ ਹੋਮ ਸਿਸਟਮ ਸੈੱਟ ਕਰਨ ਦੀ ਪ੍ਰਕਿਰਿਆ ਦਾ ਵਰਣਨ ਕਰੋ।"
    },
    {
        "prompt": "What are the benefits of renewable energy? ਨਵੀਨੀਕਰਣਯੋਗ ਊਰਜਾ ਦੇ ਫਾਇਦੇ ਕੀ ਹਨ?"
    },
    {
        "prompt": "Find the highest peak in Europe. ਯੂਰਪ ਦੀ ਸਭ ਤੋਂ ਉੱਚੀ ਚੋਟੀ ਲੱਭੋ।"
    },
    {
        "prompt": "How do you calculate the area of a trapezoid? ਤੁਸੀਂ ਇੱਕ ਟ੍ਰੇਪੇਜ਼ੋਇਡ ਦਾ ਖੇਤਰਫਲ ਕਿਵੇਂ ਗਣਨਾ ਕਰਦੇ ਹੋ?"
    },
    {
        "prompt": "List the top 10 historical landmarks in Asia. ਏਸ਼ੀਆ ਦੇ ਸਿਖਰ ਦੇ 10 ਇਤਿਹਾਸਕ ਨਿਸ਼ਾਨਿਆਂ ਦੀ ਸੂਚੀ ਦਿਓ।"
    },
    {
        "prompt": "Write a short story about a magical mirror. ਇੱਕ ਜਾਦੂਈ ਦਰਪਣ ਬਾਰੇ ਇੱਕ ਛੋਟੀ ਕਹਾਣੀ ਲਿਖੋ।"
    },
    {
        "prompt": "Explain the concept of virtual reality in education. ਸਿੱਖਿਆ ਵਿੱਚ ਵਰਚੁਅਲ ਰੀਅਲਟੀ ਦੇ ਅਸੂਲ ਨੂੰ ਸਮਝਾਓ।"
    },
    {
        "prompt": "Translate this document into Turkish. ਇਸ ਦਸਤਾਵੇਜ਼ ਨੂੰ ਤੁਰਕੀ ਵਿੱਚ ਅਨੁਵਾਦ ਕਰੋ।"
    },
    {
        "prompt": "Create a marketing plan for a new mobile app. ਇੱਕ ਨਵੇਂ ਮੋਬਾਈਲ ਐਪ ਲਈ ਮਾਰਕੀਟਿੰਗ ਯੋਜਨਾ ਬਣਾਓ।"
    },
    {
        "prompt": "Generate a list of 5 notable literary works. 5 ਪ੍ਰਸਿੱਧ ਸਾਹਿਤਕ ਕ੍ਰਿਤੀਆਂ ਦੀ ਸੂਚੀ ਬਣਾਓ।"
    },
    {
        "prompt": "Describe the process of writing a memoir. ਇੱਕ ਯਾਦਗਾਰ ਲਿਖਣ ਦੀ ਪ੍ਰਕਿਰਿਆ ਦਾ ਵਰਣਨ ਕਰੋ।"
    },
    {
        "prompt": "What are the benefits of forest conservation? ਜੰਗਲ ਸੰਰਖਣ ਦੇ ਫਾਇਦੇ ਕੀ ਹਨ?"
    },
    {
        "prompt": "Find the GDP of Canada. ਕੈਨੇਡਾ ਦਾ GDP ਲੱਭੋ।"
    },
    {
        "prompt": "How do you create a balanced meal plan? ਤੁਸੀਂ ਇੱਕ ਸੰਤੁਲਿਤ ਭੋਜਨ ਯੋਜਨਾ ਕਿਵੇਂ ਬਣਾਉਂਦੇ ਹੋ?"
    },
    {
        "prompt": "List the top 10 must-see places in South America. ਦੱਖਣੀ ਅਮਰੀਕਾ ਵਿੱਚ ਦੇਖਣ ਲਈ 10 ਜ਼ਰੂਰੀ ਸਥਾਨਾਂ ਦੀ ਸੂਚੀ ਦਿਓ।"
    },
    {
        "prompt": "Write a letter to an inspiring teacher. ਇੱਕ ਪ੍ਰੇਰਣਾਦਾਇਕ ਅਧਿਆਪਕ ਨੂੰ ਇੱਕ ਚਿੱਠੀ ਲਿਖੋ।"
    },
    {
        "prompt": "Explain the concept of artificial intelligence in healthcare. ਸਿਹਤ ਸੰਭਾਲ ਵਿੱਚ ਕ੍ਰਿਤ੍ਰਿਮ ਬੁੱਧੀ ਦੇ ਅਸੂਲ ਨੂੰ ਸਮਝਾਓ।"
    },
    {
        "prompt": "Translate this text into Italian. ਇਸ ਪਾਠ ਦਾ ਇਤਾਲਵੀ ਵਿੱਚ ਅਨੁਵਾਦ ਕਰੋ।"
    },
    {
        "prompt": "Create a business plan for a tech startup. ਇੱਕ ਤਕਨਾਲੋਜੀ ਸਟਾਰਟਅਪ ਲਈ ਵਪਾਰ ਯੋਜਨਾ ਬਣਾਓ।"
    },
    {
        "prompt": "Generate a list of 5 influential philosophers. 5 ਪ੍ਰਭਾਵਸ਼ਾਲੀ ਦਾਰਸ਼ਨਿਕਾਂ ਦੀ ਸੂਚੀ ਬਣਾਓ।"
    },
    {
        "prompt": "Describe the process of learning to play the guitar. ਗਿਟਾਰ ਵਜਾਉਣਾ ਸਿੱਖਣ ਦੀ ਪ੍ਰਕਿਰਿਆ ਦਾ ਵਰਣਨ ਕਰੋ।"
    },
    {
        "prompt": "What are the benefits of meditation for mental health? ਮਾਨਸਿਕ ਸਿਹਤ ਲਈ ਧਿਆਨ ਦੇ ਫਾਇਦੇ ਕੀ ਹਨ?"
    },
    {
        "prompt": "Find the largest rainforest in the world. ਦੁਨੀਆ ਦਾ ਸਭ ਤੋਂ ਵੱਡਾ ਵਰਖਾ ਜੰਗਲ ਲੱਭੋ।"
    },
    {
        "prompt": "How do you create a successful YouTube channel? ਤੁਸੀਂ ਇੱਕ ਸਫਲ ਯੂਟਿਊਬ ਚੈਨਲ ਕਿਵੇਂ ਬਣਾਉਂਦੇ ਹੋ?"
    },
    {
        "prompt": "List the top 10 culinary schools in the world. ਦੁਨੀਆ ਦੇ ਸਿਖਰ ਦੇ 10 ਪਕਵਾਨ ਸਕੂਲਾਂ ਦੀ ਸੂਚੀ ਦਿਓ।"
    },
    {
        "prompt": "Write a poem about a rainy day. ਇੱਕ ਬਰਸਾਤੀ ਦਿਨ ਬਾਰੇ ਇੱਕ ਕਵਿਤਾ ਲਿਖੋ।"
    },
    {
        "prompt": "Explain the concept of quantum computing. ਕਵਾਂਟਮ ਕੰਪਿਊਟਿੰਗ ਦੇ ਅਸੂਲ ਨੂੰ ਸਮਝਾਓ।"
    },
    {
        "prompt": "Translate this passage into Korean. ਇਸ ਪੈਸੇਜ ਦਾ ਕੋਰੀਅਨ ਵਿੱਚ ਅਨੁਵਾਦ ਕਰੋ।"
    },
    {
        "prompt": "Create a fitness plan for beginners. ਸ਼ੁਰੂਆਤੀਆਂ ਲਈ ਇੱਕ ਫਿਟਨੈਸ ਯੋਜਨਾ ਬਣਾਓ।"
    },
    {
        "prompt": "Generate a list of 5 influential environmentalists. 5 ਪ੍ਰਭਾਵਸ਼ਾਲੀ ਵਾਤਾਵਰਣਵਾਦੀਆਂ ਦੀ ਸੂਚੀ ਬਣਾਓ।"
    },
    {
        "prompt": "Describe the process of making homemade pasta. ਘਰੇਲੂ ਪਾਸਤਾ ਬਣਾਉਣ ਦੀ ਪ੍ਰਕਿਰਿਆ ਦਾ ਵਰਣਨ ਕਰੋ।"
    },
    {
        "prompt": "What are the benefits of learning to code? ਕੋਡਿੰਗ ਸਿੱਖਣ ਦੇ ਫਾਇਦੇ ਕੀ ਹਨ?"
    },
    {
        "prompt": "Find the smallest country in Africa. ਅਫਰੀਕਾ ਦਾ ਸਭ ਤੋਂ ਛੋਟਾ ਦੇਸ਼ ਲੱਭੋ।"
    },
    {
        "prompt": "How do you write a compelling cover letter? ਤੁਸੀਂ ਇੱਕ ਪ੍ਰਭਾਵਸ਼ਾਲੀ ਕਵਰ ਲੈਟਰ ਕਿਵੇਂ ਲਿਖਦੇ ਹੋ?"
    },
    {
        "prompt": "List the top 10 best-selling novels of all time. ਸਭ ਤੋਂ ਵੱਧ ਵਿਕਣ ਵਾਲੀਆਂ 10 ਨਾਵਲਾਂ ਦੀ ਸੂਚੀ ਦਿਓ।"
    }
]


In [15]:
custom_prompts11 = [
    {
        "prompt": "How are you? ਤੁਸੀਂ ਕਿਵੇਂ ਹੋ?"
    },
    {
        "prompt": "What is your name? ਤੁਹਾਡਾ ਨਾਂ ਕੀ ਹੈ?"
    },
    {
        "prompt": "How are you doing? ਤੁਸੀਂ ਕੀ ਕਰ ਰਹੇ ਹੋ?"
    },
    {
        "prompt": "What do you like to do for fun? ਤੁਹਾਨੂੰ ਮਜ਼ੇ ਲਈ ਕੀ ਕਰਨਾ ਪਸੰਦ ਹੈ?"
    },
    {
        "prompt": "Describe your favorite hobby. ਆਪਣਾ ਮਨਪਸੰਦ ਸ਼ੌਕ ਵਰਣਨ ਕਰੋ।"
    },
    {
        "prompt": "What is your favorite color? ਤੁਹਾਡਾ ਮਨਪਸੰਦ ਰੰਗ ਕੀ ਹੈ?"
    },
    {
        "prompt": "What is your favorite food? ਤੁਹਾਡਾ ਮਨਪਸੰਦ ਭੋਜਨ ਕੀ ਹੈ?"
    },
    {
        "prompt": "Where are you from? ਤੁਸੀਂ ਕਿੱਥੋਂ ਹੋ?"
    },
    {
        "prompt": "What is your favorite movie? ਤੁਹਾਡੀ ਮਨਪਸੰਦ ਫਿਲਮ ਕਿਹੜੀ ਹੈ?"
    },
    {
        "prompt": "Do you have any pets? ਕੀ ਤੁਹਾਡੇ ਕੋਲ ਕੋਈ ਪਾਲਤੂ ਜਾਨਵਰ ਹੈ?"
    },
    {
        "prompt": "What is your favorite book? ਤੁਹਾਡੀ ਮਨਪਸੰਦ ਕਿਤਾਬ ਕਿਹੜੀ ਹੈ?"
    },
    {
        "prompt": "Describe your dream vacation. ਆਪਣੀ ਸੁਪਨੇ ਦੀ ਛੁੱਟੀਆਂ ਦਾ ਵਰਣਨ ਕਰੋ।"
    },
    {
        "prompt": "What is your favorite season? ਤੁਹਾਡਾ ਮਨਪਸੰਦ ਮੌਸਮ ਕਿਹੜਾ ਹੈ?"
    },
    {
        "prompt": "What kind of music do you like? ਤੁਹਾਨੂੰ ਕਿਹੜੀ ਕਿਸਮ ਦਾ ਸੰਗੀਤ ਪਸੰਦ ਹੈ?"
    },
    {
        "prompt": "Do you like to cook? ਕੀ ਤੁਹਾਨੂੰ ਖਾਣਾ ਬਣਾਉਣਾ ਪਸੰਦ ਹੈ?"
    },
    {
        "prompt": "What is your favorite sport? ਤੁਹਾਡਾ ਮਨਪਸੰਦ ਖੇਡ ਕਿਹੜਾ ਹੈ?"
    },
    {
        "prompt": "Do you enjoy reading? ਕੀ ਤੁਹਾਨੂੰ ਪੜ੍ਹਨਾ ਪਸੰਦ ਹੈ?"
    },
    {
        "prompt": "What is your favorite animal? ਤੁਹਾਡਾ ਮਨਪਸੰਦ ਜਾਨਵਰ ਕਿਹੜਾ ਹੈ?"
    },
    {
        "prompt": "What do you do for a living? ਤੁਸੀਂ ਜੀਵਨ ਯਾਪਨ ਲਈ ਕੀ ਕਰਦੇ ਹੋ?"
    },
    {
        "prompt": "What is your favorite dessert? ਤੁਹਾਡਾ ਮਨਪਸੰਦ ਮਿੱਠਾ ਭੋਜਨ ਕੀ ਹੈ?"
    },
    {
        "prompt": "Do you play any instruments? ਕੀ ਤੁਸੀਂ ਕੋਈ ਸਾਜ਼ ਵਜਾਉਂਦੇ ਹੋ?"
    },
    {
        "prompt": "What is your favorite holiday? ਤੁਹਾਡਾ ਮਨਪਸੰਦ ਤਿਉਹਾਰ ਕਿਹੜਾ ਹੈ?"
    },
    {
        "prompt": "Do you have a favorite quote? ਕੀ ਤੁਹਾਡਾ ਕੋਈ ਮਨਪਸੰਦ ਉਧਰਣ ਹੈ?"
    },
    {
        "prompt": "What languages do you speak? ਤੁਸੀਂ ਕਿਹੜੀਆਂ ਭਾਸ਼ਾਵਾਂ ਬੋਲਦੇ ਹੋ?"
    },
    {
        "prompt": "Describe your perfect day. ਆਪਣਾ ਪਰਫੈਕਟ ਦਿਨ ਵਰਣਨ ਕਰੋ।"
    },
    {
        "prompt": "What is your favorite TV show? ਤੁਹਾਡਾ ਮਨਪਸੰਦ ਟੀਵੀ ਸ਼ੋ ਕਿਹੜਾ ਹੈ?"
    },
    {
        "prompt": "Do you like to travel? ਕੀ ਤੁਹਾਨੂੰ ਯਾਤਰਾ ਕਰਨੀ ਪਸੰਦ ਹੈ?"
    },
    {
        "prompt": "What is your favorite drink? ਤੁਹਾਡਾ ਮਨਪਸੰਦ ਪੀਣ ਵਾਲਾ ਪਦਾਰਥ ਕੀ ਹੈ?"
    },
    {
        "prompt": "What are your hobbies? ਤੁਹਾਡੇ ਸ਼ੌਕ ਕੀ ਹਨ?"
    },
    {
        "prompt": "What is your favorite ice cream flavor? ਤੁਹਾਡਾ ਮਨਪਸੰਦ ਆਈਸਕ੍ਰੀਮ ਸਵਾਦ ਕੀ ਹੈ?"
    },
    {
        "prompt": "Do you enjoy outdoor activities? ਕੀ ਤੁਹਾਨੂੰ ਬਾਹਰੀ ਗਤੀਵਿਧੀਆਂ ਪਸੰਦ ਹਨ?"
    },
    {
        "prompt": "What is your favorite childhood memory? ਤੁਹਾਡੀ ਮਨਪਸੰਦ ਬਚਪਨ ਦੀ ਯਾਦ ਕੀ ਹੈ?"
    },
    {
        "prompt": "What is your favorite subject in school? ਸਕੂਲ ਵਿੱਚ ਤੁਹਾਡਾ ਮਨਪਸੰਦ ਵਿਸ਼ਾ ਕੀ ਹੈ?"
    },
    {
        "prompt": "Do you prefer coffee or tea? ਤੁਹਾਨੂੰ ਕਾਫੀ ਜਾਂ ਚਾਹ ਪਸੰਦ ਹੈ?"
    },
    {
        "prompt": "What is your favorite place to visit? ਤੁਹਾਡਾ ਮਨਪਸੰਦ ਸਥਾਨ ਕਿਹੜਾ ਹੈ?"
    },
    {
        "prompt": "Do you like to exercise? ਕੀ ਤੁਹਾਨੂੰ ਵਰਜ਼ਿਸ਼ ਕਰਨੀ ਪਸੰਦ ਹੈ?"
    },
    {
        "prompt": "What is your favorite restaurant? ਤੁਹਾਡਾ ਮਨਪਸੰਦ ਰੈਸਟੋਰੈਂਟ ਕਿਹੜਾ ਹੈ?"
    },
    {
        "prompt": "Do you have a favorite artist? ਕੀ ਤੁਹਾਡਾ ਕੋਈ ਮਨਪਸੰਦ ਕਲਾਕਾਰ ਹੈ?"
    },
    {
        "prompt": "What is your favorite movie genre? ਤੁਹਾਡਾ ਮਨਪਸੰਦ ਫਿਲਮ ਸ਼ੈਲੀ ਕਿਹੜਾ ਹੈ?"
    },
    {
        "prompt": "What is your favorite thing to do on weekends? ਵੀਕਐੰਡ ਤੇ ਕਰਨ ਲਈ ਤੁਹਾਡਾ ਮਨਪਸੰਦ ਕੰਮ ਕੀ ਹੈ?"
    },
    {
        "prompt": "Do you enjoy gardening? ਕੀ ਤੁਹਾਨੂੰ ਬਾਗਬਾਨੀ ਪਸੰਦ ਹੈ?"
    },
    {
        "prompt": "What is your favorite type of cuisine? ਤੁਹਾਡਾ ਮਨਪਸੰਦ ਖਾਣਾ ਕਿਸਮ ਕਿਹੜਾ ਹੈ?"
    },
    {
        "prompt": "What is your favorite flower? ਤੁਹਾਡਾ ਮਨਪਸੰਦ ਫੁੱਲ ਕਿਹੜਾ ਹੈ?"
    },
    {
        "prompt": "What is your favorite sport to watch? ਦੇਖਣ ਲਈ ਤੁਹਾਡਾ ਮਨਪਸੰਦ ਖੇਡ ਕਿਹੜਾ ਹੈ?"
    },
    {
        "prompt": "Do you like to dance? ਕੀ ਤੁਹਾਨੂੰ ਨੱਚਣਾ ਪਸੰਦ ਹੈ?"
    },
    {
        "prompt": "What is your favorite pizza topping? ਤੁਹਾਡਾ ਮਨਪਸੰਦ ਪੀਜ਼ਾ ਟਾਪਿੰਗ ਕੀ ਹੈ?"
    },
    {
        "prompt": "Do you like to write? ਕੀ ਤੁਹਾਨੂੰ ਲਿਖਣਾ ਪਸੰਦ ਹੈ?"
    },
    {
        "prompt": "What is your favorite way to relax? ਅਰਾਮ ਕਰਨ ਲਈ ਤੁਹਾਡਾ ਮਨਪਸੰਦ ਤਰੀਕਾ ਕੀ ਹੈ?"
    },
    {
        "prompt": "What is your favorite movie character? ਤੁਹਾਡਾ ਮਨਪਸੰਦ ਫਿਲਮ ਪਾਤਰ ਕਿਹੜਾ ਹੈ?"
    }
]


# Add all the dataset (Prompt) listed above
- This is a very small dataset for toy experiments. Real datasets are vast.
- To generalize to more complex scenarios, we need a much bigger dataset.

In [16]:
custom_prompts = custom_prompts + custom_prompts2 + custom_prompts3 + custom_prompts4 + custom_prompts5 + custom_prompts6 + custom_prompts7 + custom_prompts8 + custom_prompts9 + custom_prompts10 + custom_prompts11

In [17]:
# Convert the custom prompts to a dataset
df = pd.DataFrame(custom_prompts)
dataset = Dataset.from_pandas(df)

tokenized_dataset = dataset.map(lambda samples: tokenizer(samples["prompt"], add_special_tokens=True), batched=True)

# Select a sample for training
train_sample = tokenized_dataset

# Display the tokenized samples
print(train_sample)

Map:   0%|          | 0/569 [00:00<?, ? examples/s]

Dataset({
    features: ['prompt', 'input_ids', 'attention_mask'],
    num_rows: 569
})


In [18]:
print(train_sample[0])

{'prompt': 'I want you to act as a linux terminal. ਮੈਂ ਚਾਹੁੰਦਾ ਹਾਂ ਕਿ ਤੁਸੀਂ ਇੱਕ ਲਿਨਕਸ ਟਰਮੀਨਲ ਵਜੋਂ ਕੰਮ ਕਰੋ।', 'input_ids': [44, 4026, 1152, 427, 1769, 661, 267, 104105, 28434, 17, 14715, 124220, 23868, 4627, 62475, 8174, 2011, 8710, 45171, 47792, 57755, 1028, 31828, 20016, 57769, 527], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [19]:
import peft
from peft import LoraConfig, get_peft_model, PeftModel

lora_config = LoraConfig(
    r=4,  # As bigger the R bigger the parameters to train.
    lora_alpha=1,  # A scaling factor that adjusts the magnitude of the weight matrix. Usually set to 1
    target_modules=["query_key_value"],  # You can obtain a list of target modules in the URL above.
    lora_dropout=0.05,  # Helps to avoid Overfitting.
    bias="lora_only",  # This specifies if the bias parameter should be trained.
    task_type="CAUSAL_LM"
)

peft_model = get_peft_model(foundation_model, lora_config)
print(peft_model.print_trainable_parameters())

trainable params: 589,824 || all params: 1,065,904,128 || trainable%: 0.055335558283905996
None


# Simpler training loop
- In reality, we train on more diverse datasets, and by properly scheduling learning rates, and tuning parameters based on available machine

In [20]:
import transformers
from transformers import TrainingArguments, Trainer
import os

working_dir = './'

output_directory = os.path.join(working_dir, "peft_punjabi_outputs")

# Creating the TrainingArgs
training_args = TrainingArguments(
    output_dir=output_directory,
    per_device_train_batch_size=4,  # Small batch size per GPU
    gradient_accumulation_steps=16,  # Number of steps to accumulate gradients
    learning_rate=3e-3,  # Learning rate
    num_train_epochs=100,
    fp16=True,  # Use mixed precision training if supported
    logging_dir='./logs',  # Directory for storing logs
    logging_steps=16,
)

trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=train_sample,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

trainer.train()

Step,Training Loss
16,4.177900
32,2.523600
48,2.013000
64,1.740500
80,1.546600
96,1.392100
112,1.320600
128,1.220200
144,1.175900
160,1.094700


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


TrainOutput(global_step=800, training_loss=0.9636288332939148, metrics={'train_runtime': 1441.823, 'train_samples_per_second': 39.464, 'train_steps_per_second': 0.555, 'total_flos': 4826531044306944.0, 'train_loss': 0.9636288332939148, 'epoch': 89.5104895104895})

In [21]:
# Save the model.
peft_model_path = os.path.join(output_directory, f"lora_punjabi_model")

trainer.model.save_pretrained(peft_model_path)

# Load the Model.
loaded_model = PeftModel.from_pretrained(foundation_model,
                                         peft_model_path,
                                         is_trainable=False)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


# Let us see some generations

In [22]:
# Move loaded model to GPU
loaded_model.to("cuda")

input_sentences = tokenizer("List the top 10 best-selling books of all time?", return_tensors="pt")

# Move inputs to GPU
input_sentences = {k: v.to("cuda") for k, v in input_sentences.items()}

foundational_outputs_sentence = get_outputs(loaded_model, input_sentences, max_new_tokens=10)

print(tokenizer.batch_decode(foundational_outputs_sentence, skip_special_tokens=True))


['List the top 10 best-selling books of all time? ਸਭ ਤੋਂ ਵੱਧ ਵਿਕਣ ਵਾਲੀਆਂ ਕਿਤਾਬਾਂ ਦੀ ਸੂਚੀ ਦਿਓ']


# Even when using a toy dataset: Model has learned the translations to some extend

In [29]:
# Function to test inference on sample prompts
def test_inference(model, prompts, tokenizer, max_new_tokens=10):
    model.to("cuda")
    for prompt in prompts:
        input_sentences = tokenizer(prompt, return_tensors="pt")
        input_sentences = {k: v.to("cuda") for k, v in input_sentences.items()}
        outputs = get_outputs(model, input_sentences, max_new_tokens=max_new_tokens)
        decoded_output = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        print(f"Output: {decoded_output[0]}")
        print("-" * 50)


# List of sample prompts
sample_prompts = [
    "List the top 10 best-selling books of all time?",
    "What languages do you speak?",
    "What is your favorite holiday?"
]

# Run inference on sample prompts
test_inference(loaded_model, sample_prompts, tokenizer, max_new_tokens=10)

Output: List the top 10 best-selling books of all time? ਸਭ ਤੋਂ ਵੱਧ ਵਿਕਣ ਵਾਲੀਆਂ ਕਿਤਾਬਾਂ ਦੀ ਸੂਚੀ ਦਿਓ
--------------------------------------------------
Output: What languages do you speak? ਤੁਸੀਂ ਕਿਹੜੀਆਂ ਭਾਸ਼ਾ ਬੋਲਦੇ ਹੋ?? ਕੀ
--------------------------------------------------
Output: What is your favorite holiday? आपका ਮਨਪਸੰਦ ਤਿਉਹਾਰ ਕਿਹੜਾ ਹੈ?
--------------------------------------------------


# Run on new sentences which were not seen in training

In [37]:
# List of sample prompts
sample_prompts = [
    "I was born in 2010.",
    "I want to learn.",
    "I am not going to school today."
]

# Run inference on sample prompts
test_inference(loaded_model, sample_prompts, tokenizer, max_new_tokens=10)

Output: I was born in 2010. ਮੈਂ 2010 ਵਿੱਚ ਹੋਇਆ ਹੈ। ਇੱਕ ਹੋ ਗਿਆ ਹੈ
--------------------------------------------------
Output: I want to learn. ਮੈਂ ਮੰਗ ਕਰ ਰਹੇ ਹਾਂ। ਮੈਂ ਚਾਹੁੰਦਾ ਹਾਂ।
--------------------------------------------------
Output: I am not going to school today. ਅੱਜ ਸਕੂਲ ਨਾ ਆਉਣ ਦਾ ਦਾਅਵਾ ਨਾ ਕਰਨ। ਨਾ
--------------------------------------------------
